In [1]:
import os as os
import pandas as pd
import numpy as np
import itertools
from openpyxl import load_workbook
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_columns', None)
from pathlib import Path
from openpyxl import load_workbook
import re


## Define directory to point to for source data

In [2]:
# Get current notebook directory
notebook_dir = Path.cwd()

# Move up to project root (NBA-PostgreSQL-Database) and then to source_data
project_root = notebook_dir.parent
source_data_path = project_root / 'raw_extracted_source_data'
source_data_path

WindowsPath('C:/Users/Carey/NBA-PostgreSQL-Database/raw_extracted_source_data')

In [3]:
# Get current notebook directory
notebook_dir = Path.cwd()

# Move up to project root (NBA-PostgreSQL-Database) and then to source_data
project_root = notebook_dir.parent
final_data_for_upload_path = project_root / 'transformed_data_for_database_load'
final_data_for_upload_path

WindowsPath('C:/Users/Carey/NBA-PostgreSQL-Database/transformed_data_for_database_load')

# TABLE OF CONTENTS


# Stage 1.) Create standardized validation and cleaning functions
        
        1.) Validate that no yearly files are the EXACT same
        2.) Validate that all yearly files being combined have the same column structure
        3.) Validate compilation. Ensure the sum of record counts of yearly results = record count of combined dataframe
        4.) Validate data completeness. Make sure we have data for every year in the range 2004-2023
        5.) Clean player/team names of symbols that denote something in the original source data
        

# Stage 2.) Compile dataframes from yearly source files. Validate and do preliminary cleaning

## 2.1) PLAYER STATS:

#### Player Regular Season Stats Dataframes
* [Data Source 1. BASIC PLAYER STATS IN REGULAR SEASON](#Step1)
        
* [Data Source 2. ADVANCED PLAYER STATS IN REGULAR SEASON](#Step2)
        - Validate player list of advanced stats = player list of basic stats for each year

* [Data Source 3. PLAYER SHOOTING STATS IN REGULAR SEASON](#Step3)
        - Validate player list of shooting stats = player list of basic stats for each year
        
* [Data Source 4. PLAYER PLAY-BY-PLAY STATS IN REGULAR SEASON](#Step4)
        - Validate player list of shooting stats = player list of basic stats for each year

#### Player Playoff Stats Dataframes

* [Data Source 5. BASIC PLAYER STATS IN PLAYOFFS](#Step5)

* [Data Source 6. ADVANCED PLAYER STATS IN PLAYOFFS](#Step6)
        - Validate player list of basic stats in playoffs = player list of advanced stats in playoffs for each year
        
* [Data Source 7. PLAYER SHOOTING STATS IN PLAYOFFS](#Step7)
        - Validate player list of shooting stats = player list of basic stats for each year
        
* [Data Source 8. PLAYER PLAY-BY-PLAY STATS IN PLAYOFFS](#Step8)
        - Validate player list of shooting stats = player list of basic stats for each year        
        
#### Player Awards (e.g. MVP) Dataframes
* [Data Source 9. AWARD VOTING HISTORY](#Step9)
        - Validate manual extraction accuracy by ensuring award winners actually had stats in the year they won the award
* [Data Source 10. FINALS MVP](#Step10)
        - Validate manual extraction accuracy by ensuring award winners actually had stats in the year they won the award
        
## 2.2) TEAM STATS: 

#### Team Regular Season Stats Dataframes
* [Data Source 11. BASIC TEAM STATS](#Step11)
* [Data Source 12. OPPONENT STATS AGAINST TEAM](#Step12)
* [Data Source 13. ADVANCED TEAM STATS](#Step13)
* [Data Source 14. TEAM SHOOTING STATS](#Step14)
* [Data Source 15, OPPONENT SHOOTING AGAINST TEAM STATS](#Step15)

#### Team Regular Season Standings Dataframe
* [Data Source 16. REGULAR SEASON TEAM STANDINGS](#Step22)

#### Team Playoff Stats Dataframes
* [Data Source 17. BASIC TEAM STATS](#Step16)
* [Data Source 18. OPPONENT STATS AGAINST TEAM](#Step17)
* [Data Source 19. ADVANCED TEAM STATS](#Step18)
* [Data Source 20. TEAM SHOOTING STATS](#Step19)
* [Data Source 21. OPPONENT SHOOTING AGAINST TEAM STATS](#Step20)

#### Team Playoff Standings Dataframe
* [Data Source 22. PLAYOFFS TEAM STANDINGS](#Step21)
  
# Stage 3.) Extensively clean and validate all dataframes as prep for joins
* [Step 1. Standardardize name of "Player Unique ID" column for all player-related dataframes](#Step23)
        - This is necessary because the name of the columns were one of "-9999","Player-additional", or "-additional"
* [Step 2. Standardardize name of "Team Name" column for all player-related dataframes](#Step24)
        - This is necessary because the name of the columns were one of "Tm" or "Team"
* [Step 3. Address duplicate records for regular season stats where players got traded mid season](#Step25)
        - For players that got traded mid-season, keep team value as the team they were traded to last before drop
        - Drop duplicate records for players that got traded
        - After drop, validate only one player record exists in each regular season player-related dataframe
        - After drop, validate all regular season player stat dataframes are the same length
* [Step 4. Validate we don't have the same issue for playoff stats for players](#Step26) 
        - Validate that there aren't multiple player records in playoffs (players can't be traded, so shouldn't)
        - Validate all playoff player stat dataframes are the same length as expected
* [Step 5. Normalize team names across years where teams change location or name](#Step27) 
        - Examples include Charlotte Bobcats -> Charlotte Hornets. Or Seattle SuperSonics to OKC Thunder
        
# Stage 4.) Create keys for table joins across all dataframes

* [Key creation 1. Create Player-unique-id-year-concat to enable joins between player tables](#Step28)
        - Example of a key for a Giannis record in 2019 would be "antetgi01-2019"
* [Key creation 2. Create Team-name-year-concat to enable joins between all tables](#Step29) 
        - Example of a key for a Toronto Raptors record in 2019 would be "Toronto Raptors-2019"
        
# Stage 5.) Change each dataframe's column names to be both more understandable and unique between tables once merged 
* [Player Related Stats. Change column names](#Step30)
        - Include table name in column name so that when we join tables we can differentiate same named columns 
            - For example, there may be 3P% across multiple tables. Change 3P% to "Regular season player 3 Point %"
* [Team Related Stats. Change column names](#Step31)
        - Include table name in column name so that when we join tables we can differentiate same named columns 
            - For example, there may be team 3P% across multiple tables. Change 3P% to "Team Playoff 3 Point %"
            
# Stage 6.) Do some feature engineering to aid machine learning in Stage 8+ and refine dataframes further before merge
* [Create indicators for each type of qualified* player stat](#Step99)
        - Example: create column to indicate a player has played more than 50 games*
* [Create ranked stats columns for key stats so you have performance relative to peers](#Step99)
        - Example: create column that is the rank of each player's Points Per Game (e.g. top scorer = 1)
* [Encode variables like "Position"](#Step99)
* [Perform final cleaning by excluding redundant columns prior to merging tables](#Step99)
        - Examples of redundant columns: Player Name, Team Name, Year
       
# Stage 7.) Join all REGULAR SEASON player stats, player awards, and team stats tables together into a single, combined dataframe. The machine learning models will train on this combined dataframe. 


## PLAYER STATS MERGED DATAFRAMES

* [Merged Table 1. MERGE ALL REGULAR SEASON PLAYER STATS. Join the following tables together:](#Step32)
        - Basic Player Stats (Regular Season)
        - Advanced Player Stats (Regular Season)
        - Player Shooting Stats (Regular Season)
        - Player Play by Play Stats (Regular Season)
        
* [Merged Table 2. MERGE ALL REGULAR  TEAM STATS. Join the following tables together:](#Step33)
        - Basic Team Stats (Regular Season)
        - Basic Opponent Team Stats (Regular Season)
        - Advanced Team Stats (Regular Season)
        - Team Shooting Stats (Regular Season)
        - Team Opponent Shooting Stats (Regular Season)
        - Team Regular Season Record
        
## Remove 2024 data and store. This data will be used separately for predicting, not training

# Stage 8.) Create machine learning models to predict vote share % for each end of season player award
        
        Standard Procedures:
        - 1.) Merge player stats, team stats, and award stat table to have final data set
        - 2.) Split data into training and testing data sets
        - 3.) Evaluate model predictive power
        - 4.) Identify which features were most important to model
        - 5.) Try to simplify/refine model by filtering down to subset of features 
        - 6.) Use final model to populate and store the model-predicted vote % for every player-season
        - 7.) Use final model to predict 2024 award winner



* [MVP Award Prediction](#Step34)
* [DPOY Award Prediction](#Step35)
* [6MAN Award Prediction](#Step36)
        - Filtered down training/testing data to just 6MAN eligible candidates (>60 games played, <35 games started)
* [ROY Award Prediction](#Step37)
        - Load in rookie data spanning 2003-2024 so that we can flag which player-seasons qualified as rookie seasons
        - Filtered down training/testing data to just rookies        
* [MIP Award Prediction](#Step38)
        - Create delta columns to store changes in stats between seasons to help model understand "improvement"
* [2024 Playoff Prediction](#Step39)
        - The results of this exercise were unfortunately very boring. Complex models just barely edged out purely using regular season team record. Turns out regular seasoning winning is really the only important predictor of post season winning 
          
# Stage 9.) Compile all dataframes into one
* [Merged Table 3. MERGE ALL PLAYER AWARDS. Join the following tables together:](#Step40)
        - MVP Vote History
        - ROY Vote History
        - All NBA Vote History
        - All Defense History
        - MIP Vote History
        - Six Man Vote History        
        - DPOY Vote History
        - Finals MVP Vote History
* [Merged Table 4. MERGE ALL PLAYOFF PLAYER STATS. Join the following tables together:](#Step41)
        - Basic Player Stats (Playoffs)
        - Advanced Player Stats (Playoffs)
        - Player Shooting Stats (Playoffs)
        - Player Play by Play Stats (Playoffs)
        
* [Merged Table 5. MERGE ALL PLAYOFF TEAM STATS. Join the following tables together:](#Step42)
        - Basic Team Stats (Playoffs)
        - Basic Opponent Team Stats (Playoffs)
        - Advanced Team Stats (Playoffs)
        - Team Shooting Stats (Playoffs)
        - Team Opponent Shooting Stats (Playoffs)    
        
* [Merged Table 6. MERGE ALL TEAM RELATED DATA. Join the following tables together:](#Step43)  
        - ALL REGULAR SEASON TEAM STATS (created above by merging)
        - ALL PLAYOFF TEAM STATS (created above by merging)
        - Team Playoff Record         
        
* [Merged Table 7. MERGE ALL PLAYER RELATED DATA. Join the following tables together:](#Step44)  
        - ALL REGULAR SEASON PLAYER STATS (created above by merging)
        - ALL PLAYOFF PLAYER STATS (created above by merging)
        - All PLAYER AWARDS (created above by merging)
        
* [Merged Table 8. MERGE ALL PLAYER AND TEAM RELATED DATA INTO ONE ULTIMATE DATAFRAME](#Step45)  
        - ALL TEAM RELATED DATA (created above)
        - ALL PLAYER RELATED DATA (created above)
        
# Stage 10.) Export final dataframes of interest

* [Special combined team stats containing league average data](#Step46)  
* [ULTIMATE FINAL COMBINED DATAFRAME](#Step47)  
               

# Stage 1.) Create list of validation functions

In [4]:
def create_list_of_dataframes_for_yearly_stats(folder_path,header_row_value):

    master_df_list = []

    cumulative_row_counter = 0

    for file_name in os.listdir(folder_path):
        
        if file_name == 'desktop.ini':
            
            continue
        
        # Assign season value
        year = file_name.split(' ')[0]

        # Create dataframe name
        dataframe_name = f'_{file_name.split(".")[0].lower().replace(" ","_")}_df'

        # Store dataframe globally with specified name
        globals()[dataframe_name] = pd.read_csv(f'{folder_path}\\{file_name}',header=header_row_value)

        # Add a Year Column and assign to year
        globals()[dataframe_name]['Year'] = int(year)

        # Append to master_df_list
        master_df_list.append(globals()[dataframe_name]) 

        # Cumulative_row_counter
        cumulative_row_counter = cumulative_row_counter + len(globals()[dataframe_name])
    
    globals()['master_df_list'] = master_df_list
    globals()['cumulative_row_counter'] = cumulative_row_counter
    
    return master_df_list[0].head(1)
    

In [5]:
def check_all_unique_dataframes(dfs,column_to_ignore):
    num_dfs = len(dfs)
    identical_pairs = []
    
    # Dataframes with column removed
    list_of_dataframes_edited = []
    
    # Remove column to ignore
    for i in range(num_dfs):
        edited_df = dfs[i].drop(column_to_ignore,axis=1)
        list_of_dataframes_edited.append(edited_df)

    # Compare and append
    for i in range(num_dfs):
        for j in range(i+1, num_dfs):
            if list_of_dataframes_edited[i].equals(list_of_dataframes_edited[j]):
                identical_pairs.append((i, j))
                
    if len(identical_pairs)>0:
        raise Exception(f'MATCHING DATAFRAMES FOUND - {identical_pairs}. This is likely the result of accidentally copying the wrong years data while creating yearly data files')  

In [6]:
def validate_column_structure_is_same(list_of_dataframes):
    
    first_dataframe_column_structure = list(list_of_dataframes[0].columns)
    first_dataframe_column_structure

    for dataframe in list_of_dataframes:
        if list(dataframe.columns)!=first_dataframe_column_structure:
            raise Exception(f"{dataframe} columns don't match. Review files to ensure they're all the same structure and that a file is not misplaced")

In [7]:
def concat_dataframes_and_validate_and_reset_index_and_sort(desired_name_of_output_dataframe,list_of_dataframes_to_concat,length_value_to_use_to_validate):
    
    globals()[desired_name_of_output_dataframe] = pd.concat(list_of_dataframes_to_concat)

    # Verify that total record count of master dataframe matches the cumulative record count of the original dataframes
    if len(globals()[desired_name_of_output_dataframe])!=length_value_to_use_to_validate:
        raise Exception('Record count of master dataframe does not match cumulative count of component dataframes')

    # Reset indices and sort
    globals()[desired_name_of_output_dataframe] = globals()[desired_name_of_output_dataframe].sort_values('Year',ascending=False)
    globals()[desired_name_of_output_dataframe].reset_index(drop=True,inplace=True)
    globals()[desired_name_of_output_dataframe].head(1)

In [8]:
def validate_completeness_of_data(dataframe,beginning_year,ending_year_plus_one): 

    # Validate that every year contains data
    set_of_years_in_dataframe = set(dataframe['Year'])
    set_of_years_in_range = set(range(2004,2024))

    years_not_in_dataframe = list(set_of_years_in_range-set_of_years_in_dataframe)

    if len(years_not_in_dataframe)>0:
        raise Exception(f'Missing data for the following years: {years_not_in_dataframe}')

In [9]:
def clean_column_values(dataframe,column_to_clean):
    
    # Clean player names of symbols like asterisks and names with '.'
    dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
    dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')

In [10]:
def validate_dataframes_using_column_values(dataframe_to_compare,column_to_compare,dataframe_to_compare_against,column_to_compare_against):
    
    for year in dataframe_to_compare['Year'].unique():
        
        # Filter df of interest on year
        dataframe_to_compare = dataframe_to_compare.loc[dataframe_to_compare['Year']==year]

        # Grab list of players for year
        unique_values_of_column_to_compare = set(dataframe_to_compare[column_to_compare])

        # Filter basic yearly player stats df on year
        dataframe_to_compare_against = dataframe_to_compare_against.loc[dataframe_to_compare_against['Year']==year]

        # Grab list of players that played in year
        unique_values_of_column_to_compare_against = set(dataframe_to_compare_against[column_to_compare_against])

        # Determine difference
        unique_values_not_found = list(unique_values_of_column_to_compare - unique_values_of_column_to_compare_against)

        # Raise error if players are found to not have played in year
        if len(unique_values_not_found)>0:
             raise Exception(f'For {year}, {unique_values_not_found} not found in dataframe used to compare against. Likely issue of loading the wrong years data.')


# Stage 2.) Compile PLAYER related dataframes from yearly source files

## Regular Season Player Stats

### BASIC PLAYER STATS IN REGULAR SEASON. Compile yearly data into one dataframe
 <a class="anchor" id="Step1"></a>

#### Create dataframes of yearly data

In [11]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Player Stats\\Regular season\\NBA Basic Player Stats',0)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional,Year
0,1,Shareef Abdur-Rahim,PF,27,TOT,85,56,31.6000,5.9000,12.4000,0.4750,0.1000,0.4000,0.2650,5.8000,12.0000,0.4820,0.4800,4.4000,5.0000,0.8690,2.2000,5.3000,7.5000,2.0000,0.8000,0.4000,2.2000,2.6000,16.3000,abdursh01,2004


###### Validate that all of the yearly data files contain different data

In [12]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [13]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [14]:
concat_dataframes_and_validate_and_reset_index_and_sort('basic_player_stats_regular_season_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [15]:
validate_completeness_of_data(basic_player_stats_regular_season_df,2004,2024)

###### Clean player names

In [16]:
clean_column_values(basic_player_stats_regular_season_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [17]:
# N/A

### ADVANCED PLAYER STATS IN REGULAR SEASON. Compile yearly data into one dataframe
 <a class="anchor" id="Step2"></a>

#### Create dataframes of yearly data

In [18]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Player Stats\\Regular season\\NBA Advanced Player Stats',0)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player-additional,Year
0,1,Shareef Abdur-Rahim,PF,27,TOT,85,2684,19.9000,0.5570,0.0320,0.4070,8.2000,19.0000,13.6000,12.3000,1.4000,1.0000,12.9000,24.4000,NaN,6.1000,2.0000,8.1000,0.1450,NaN,2.3000,-0.7000,1.6000,2.4000,abdursh01,2004


###### Validate that all of the yearly data files contain different data

In [19]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [20]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [21]:
concat_dataframes_and_validate_and_reset_index_and_sort('advanced_player_stats_in_regular_season_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [22]:
validate_completeness_of_data(advanced_player_stats_in_regular_season_df,2004,2024)

###### Clean player names

In [23]:
clean_column_values(advanced_player_stats_in_regular_season_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [24]:
validate_dataframes_using_column_values(advanced_player_stats_in_regular_season_df,'Player-additional',basic_player_stats_regular_season_df,'Player-additional')

### PLAYER SHOOTING IN REGULAR SEASON. Compile yearly data into one dataframe
 <a class="anchor" id="Step3"></a>

#### Create dataframes of yearly data

In [25]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Player Stats\\Regular season\\NBA Player Shooting Stats',1)

,Rk,Player,Pos,Age,Tm,G,MP,FG%,Dist.,Unnamed: 9,2P,0-3,3-10,10-16,16-3P,3P,Unnamed: 16,2P.1,0-3.1,3-10.1,10-16.1,16-3P.1,3P.1,Unnamed: 23,2P.2,3P.2,Unnamed: 26,%FGA,#,Unnamed: 29,%3PA,3P%,Unnamed: 32,Att.,#.1,-9999,Year
0,1,Shareef Abdur-Rahim,PF,27,TOT,85,2684,0.4750,7.8000,NaN,0.9680,0.3570,0.3190,0.1160,0.1760,0.0320,NaN,0.4820,0.5850,0.4260,0.4260,0.4140,0.2650,NaN,0.5390,0.8890,NaN,0.0690,71,NaN,0.2060,0.4290,NaN,0,0,abdursh01,2004


###### Validate that all of the yearly data files contain different data

In [26]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [27]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [28]:
concat_dataframes_and_validate_and_reset_index_and_sort('player_shooting_in_regular_season_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [29]:
validate_completeness_of_data(player_shooting_in_regular_season_df,2004,2024)

###### Clean player names

In [30]:
clean_column_values(advanced_player_stats_in_regular_season_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [31]:
validate_dataframes_using_column_values(player_shooting_in_regular_season_df,'-9999',basic_player_stats_regular_season_df,'Player-additional')

### PLAYER PLAY-BY-PLAY STATS IN REGULAR SEASON. Compile yearly data into one dataframe
 <a class="anchor" id="Step4"></a>

#### Create dataframes of yearly data

In [32]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Player Stats\\Regular season\\NBA Player Play by Play Stats',1)

,Rk,Player,Pos,Age,Tm,G,MP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPass,LostBall,Shoot,Off.,Shoot.1,Off..1,PGA,And1,Blkd,-9999,Year
0,1,Shareef Abdur-Rahim,PF,27,TOT,85,2684,NaN,NaN,6%,93%,1%,-2.5000,2.3000,56,72,95,29,185,NaN,390,31,94,abdursh01,2004


###### Validate that all of the yearly data files contain different data

In [33]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [34]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [35]:
concat_dataframes_and_validate_and_reset_index_and_sort('player_play_by_play_in_regular_season_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [36]:
validate_completeness_of_data(player_play_by_play_in_regular_season_df,2004,2024)

###### Clean player names

In [37]:
clean_column_values(player_play_by_play_in_regular_season_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [38]:
validate_dataframes_using_column_values(player_play_by_play_in_regular_season_df,'-9999',basic_player_stats_regular_season_df,'Player-additional')

## Player Playoff Stats

### BASIC PLAYER STATS IN PLAYOFFS. Compile yearly data into one dataframe
 <a class="anchor" id="Step5"></a>

#### Create dataframes of yearly data

In [39]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Player Stats\\Playoffs\\NBA Basic Player Stats in Playoffs',0)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional,Year
0,1,Malik Allen,PF,25,MIA,10,0,13.8000,2.2000,4.9000,0.4490,0.0000,0.0000,NaN,2.2000,4.9000,0.4490,0.4490,0.6000,0.9000,0.6670,1.2000,1.8000,3.0000,0.4000,0.2000,0.9000,0.4000,2.1000,5.0000,allenma01,2004


###### Validate that all of the yearly data files contain different data

In [40]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [41]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [42]:
concat_dataframes_and_validate_and_reset_index_and_sort('player_basic_playoff_stats_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [43]:
validate_completeness_of_data(player_basic_playoff_stats_df,2004,2024)

###### Clean player names

In [44]:
clean_column_values(player_basic_playoff_stats_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [45]:
# N/A

### ADVANCED PLAYER STATS IN PLAYOFFS. Compile yearly data into one dataframe
 <a class="anchor" id="Step6"></a>

#### Create dataframes of yearly data

In [46]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Player Stats\\Playoffs\\NBA Advanced Player Stats in Playoffs',0)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player-additional,Year
0,1,Malik Allen,PF,25,MIA,10,138,16.1000,0.4720,0.0000,0.1840,10.0000,15.0000,12.5000,6.1000,0.8000,5.6000,7.0000,19.8000,NaN,0.2000,0.2000,0.4000,0.1310,NaN,-0.8000,1.0000,0.2000,0.1000,allenma01,2004


###### Validate that all of the yearly data files contain different data

In [47]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [48]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [49]:
concat_dataframes_and_validate_and_reset_index_and_sort('player_advanced_stats_in_playoffs_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [50]:
validate_completeness_of_data(player_advanced_stats_in_playoffs_df,2004,2024)

###### Clean player names

In [51]:
clean_column_values(player_advanced_stats_in_playoffs_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [52]:
validate_dataframes_using_column_values(player_advanced_stats_in_playoffs_df,'Player-additional',player_basic_playoff_stats_df,'Player-additional')

### PLAYER SHOOTING STATS IN PLAYOFFS. Compile yearly data into one dataframe
 <a class="anchor" id="Step7"></a>

#### Create dataframes of yearly data

In [53]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Player Stats\\Playoffs\\NBA Player Shooting',1)

,Rk,Player,Pos,Age,Tm,G,MP,FG%,Dist.,Unnamed: 9,2P,0-3,3-10,10-16,16-3P,3P,Unnamed: 16,2P.1,0-3.1,3-10.1,10-16.1,16-3P.1,3P.1,Unnamed: 23,2P.2,3P.2,Unnamed: 26,%FGA,#,Unnamed: 29,%3PA,3P%,Unnamed: 32,Att.,#.1,-9999,Year
0,1,Malik Allen,PF,25,MIA,10,138,0.4490,10.6000,NaN,1.0000,0.2040,0.2450,0.1840,0.3670,0.0000,NaN,0.4490,0.5000,0.6670,0.2220,0.3890,NaN,NaN,0.5910,NaN,NaN,0.0000,0,NaN,NaN,NaN,NaN,0,0,allenma01,2004


###### Validate that all of the yearly data files contain different data

In [54]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [55]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [56]:
concat_dataframes_and_validate_and_reset_index_and_sort('player_shooting_stats_in_playoffs_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [57]:
validate_completeness_of_data(player_shooting_stats_in_playoffs_df,2004,2024)

###### Clean player names

In [58]:
clean_column_values(player_shooting_stats_in_playoffs_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [59]:
validate_dataframes_using_column_values(player_shooting_stats_in_playoffs_df,'-9999',player_basic_playoff_stats_df,'Player-additional')

### PLAYER PLAY-BY-PLAY STATS IN PLAYOFFS. Compile yearly data into one dataframe
 <a class="anchor" id="Step8"></a>

#### Create dataframes of yearly data

In [60]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Player Stats\\Playoffs\\NBA Player Play by Play',1)

,Rk,Player,Pos,Age,Tm,G,MP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPass,LostBall,Shoot,Off.,Shoot.1,Off..1,PGA,And1,Blkd,-9999,Year
0,1,Malik Allen,PF,25,MIA,10,138,NaN,NaN,2%,27%,71%,8.4000,10.6000,1,2,9,1,4,NaN,8,1,4,allenma01,2004


###### Validate that all of the yearly data files contain different data

In [61]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [62]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [63]:
concat_dataframes_and_validate_and_reset_index_and_sort('player_play_by_play_stats_in_playoffs_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [64]:
validate_completeness_of_data(player_play_by_play_stats_in_playoffs_df,2004,2024)

###### Clean player names

In [65]:
clean_column_values(player_play_by_play_stats_in_playoffs_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [66]:
validate_dataframes_using_column_values(player_play_by_play_stats_in_playoffs_df,'-9999',player_basic_playoff_stats_df,'Player-additional')

## AWARD VOTING HISTORY. For each award, compile yearly data into one dataframe
 <a class="anchor" id="Step9"></a>

### MVP. Compile yearly data into one dataframe

#### Create dataframes of yearly data

In [67]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\NBA Player Awards\\MVP',1)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,-9999,Year
0,1,Kevin Garnett,27,MIN,120.0000,1219.0000,1230,0.9910,82,39.4000,24.2000,13.9000,5.0000,1.5000,2.2000,0.4990,0.2560,0.7910,18.3000,0.2720,garneke01,2004


###### Validate that all of the yearly data files contain different data

In [68]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [69]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [70]:
concat_dataframes_and_validate_and_reset_index_and_sort('mvp_vote_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [71]:
validate_completeness_of_data(mvp_vote_df,2004,2024)

###### Clean player names

In [72]:
clean_column_values(mvp_vote_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [73]:
validate_dataframes_using_column_values(mvp_vote_df,'-9999',basic_player_stats_regular_season_df,'Player-additional')

### ROY. Compile yearly data into one dataframe

#### Create dataframes of yearly data

In [74]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\NBA Player Awards\\ROY',1)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,-9999,Year
0,1,LeBron James,19,CLE,78.0000,508.0000,590,0.8610,79,39.5000,20.9000,5.5000,5.9000,1.6000,0.7000,0.4170,0.2900,0.7540,5.1000,0.0780,jamesle01,2004


###### Validate that all of the yearly data files contain different data

In [75]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [76]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [77]:
concat_dataframes_and_validate_and_reset_index_and_sort('roy_vote_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [78]:
validate_completeness_of_data(roy_vote_df,2004,2024)

###### Clean player names

In [79]:
clean_column_values(roy_vote_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [80]:
validate_dataframes_using_column_values(roy_vote_df,'-9999',basic_player_stats_regular_season_df,'Player-additional')

### ALL NBA. Compile yearly data into one dataframe

#### Create dataframes of yearly data

In [81]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\NBA Player Awards\\ALL NBA',1)

,# Tm,Pos,Player,Age,Tm,Pts Won,Pts Max,Share,1st Tm,2nd Tm,3rd Tm,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,-9999,Year
0,1st,F,Kevin Garnett,27,MIN,615,615,1.0000,123,0.0000,0.0000,82,39.4000,24.2000,13.9000,5.0000,1.5000,2.2000,0.4990,0.2560,0.7910,18.3000,0.2720,garneke01,2004


###### Validate that all of the yearly data files contain different data

In [82]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [83]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [84]:
concat_dataframes_and_validate_and_reset_index_and_sort('all_nba_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [85]:
validate_completeness_of_data(all_nba_df,2004,2024)

###### Clean player names

In [86]:
clean_column_values(all_nba_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [87]:
validate_dataframes_using_column_values(all_nba_df,'-9999',basic_player_stats_regular_season_df,'Player-additional')

### ALL DEFENSE. Compile yearly data into one dataframe

#### Create dataframes of yearly data

In [88]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\NBA Player Awards\\ALL Defense',1)

,# Tm,Pos,Player,Age,Tm,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,DWS,DBPM,DRtg,-9999,Year
0,1st,F,Metta World Peace,24,IND,54,58,0.9310,73,37.2000,18.3000,5.3000,3.7000,2.1000,0.7000,0.4210,0.3100,0.7330,8.0000,0.1410,5.2000,1.9000,96,artesro01,2004


###### Validate that all of the yearly data files contain different data

In [89]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [90]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [91]:
concat_dataframes_and_validate_and_reset_index_and_sort('all_defense_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [92]:
validate_completeness_of_data(all_defense_df,2004,2024)

###### Clean player names

In [93]:
clean_column_values(all_defense_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [94]:
validate_dataframes_using_column_values(all_defense_df,'-9999',basic_player_stats_regular_season_df,'Player-additional')

### MIP. Compile yearly data into one dataframe

#### Create dataframes of yearly data

In [95]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\NBA Player Awards\\MIP',1)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,-9999,Year
0,1,Zach Randolph,22,POR,59.0000,379.0000,605,0.6260,81,37.9000,20.1000,10.5000,2.0000,0.8000,0.5000,0.4850,0.2000,0.7610,7.1000,0.1110,randoza01,2004


###### Validate that all of the yearly data files contain different data

In [96]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [97]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [98]:
concat_dataframes_and_validate_and_reset_index_and_sort('mip_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [99]:
validate_completeness_of_data(mip_df,2004,2024)

###### Clean player names

In [100]:
clean_column_values(mip_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [101]:
validate_dataframes_using_column_values(mip_df,'-9999',basic_player_stats_regular_season_df,'Player-additional')

### 6th Man. Compile yearly data into one dataframe

#### Create dataframes of yearly data

In [102]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\NBA Player Awards\\6MAN',1)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,-9999,Year
0,1,Antawn Jamison,27,DAL,43.0000,338.0000,600,0.5630,82,29.0000,14.8000,6.3000,0.9000,1.0000,0.4000,0.5350,0.4000,0.7480,9.0000,0.1830,jamisan01,2004


###### Validate that all of the yearly data files contain different data

In [103]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [104]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [105]:
concat_dataframes_and_validate_and_reset_index_and_sort('six_man_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [106]:
validate_completeness_of_data(six_man_df,2004,2024)

###### Clean player names

In [107]:
clean_column_values(six_man_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [108]:
validate_dataframes_using_column_values(six_man_df,'-9999',basic_player_stats_regular_season_df,'Player-additional')

### DPOY. Compile yearly data into one dataframe

#### Create dataframes of yearly data

In [109]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\NBA Player Awards\\DPOY',1)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,DWS,DBPM,DRtg,-9999,Year
0,1,Metta World Peace,24,IND,80.0000,476.0000,605,0.7870,73,37.2000,18.3000,5.3000,3.7000,2.1000,0.7000,0.4210,0.3100,0.7330,8.0000,0.1410,5.2000,1.9000,96,artesro01,2004


###### Validate that all of the yearly data files contain different data

In [110]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [111]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [112]:
concat_dataframes_and_validate_and_reset_index_and_sort('dpoy_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [113]:
validate_completeness_of_data(dpoy_df,2004,2024)

###### Clean player names

In [114]:
clean_column_values(dpoy_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [115]:
validate_dataframes_using_column_values(dpoy_df,'-9999',basic_player_stats_regular_season_df,'Player-additional')

## LOAD IN FINALS MVP AND STATS

In [116]:
# Load in data
finals_mvp = pd.read_csv(f'{source_data_path}\\NBA Player Awards\\Finals MVP Results and Stats.txt',header=1)

# Convert season to year
finals_mvp.rename({'Season':'Year'},axis=1,inplace=True)
finals_mvp['Year'] = [str(int(year.split('-')[1]) + 2000) if len(year.split('-')[1]) == 2 else year.split('-')[1] 
                     for year in finals_mvp['Year']]

# Filter on just years of interest
list_of_years = list(range(2003,2024))
finals_mvp['Year'] = finals_mvp['Year'].astype(int)
finals_mvp = finals_mvp.loc[finals_mvp['Year'].isin(list_of_years)]

finals_mvp.head(30)

,Year,Lg,Player,Age,Tm,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,-9999
0,2023,NBA,Nikola Jokić,27,DEN,5,41.2000,30.2000,14.0000,7.2000,0.8000,1.4000,0.5830,0.4210,0.8380,jokicni01
1,2022,NBA,Stephen Curry,33,GSW,6,37.5000,31.2000,6.0000,5.0000,2.0000,0.2000,0.4820,0.4370,0.8570,curryst01
2,2021,NBA,Giannis Antetokounmpo,26,MIL,6,39.8000,35.2000,13.2000,5.0000,1.2000,1.8000,0.6180,0.2000,0.6590,antetgi01
3,2020,NBA,LeBron James,35,LAL,6,39.3000,29.8000,11.8000,8.5000,1.2000,0.5000,0.5910,0.4170,0.6670,jamesle01
4,2019,NBA,Kawhi Leonard,27,TOR,6,40.5000,28.5000,9.8000,4.2000,2.0000,1.2000,0.4340,0.3570,0.9060,leonaka01
5,2018,NBA,Kevin Durant,29,GSW,4,41.3000,28.8000,10.8000,7.5000,0.8000,2.3000,0.5260,0.4090,0.9630,duranke01
6,2017,NBA,Kevin Durant,28,GSW,5,39.8000,35.2000,8.2000,5.4000,1.0000,1.6000,0.5560,0.4740,0.9270,duranke01
7,2016,NBA,LeBron James,31,CLE,7,41.7000,29.7000,11.3000,8.9000,2.6000,2.3000,0.4940,0.3710,0.7210,jamesle01
8,2015,NBA,Andre Iguodala,31,GSW,6,37.0000,16.3000,5.8000,4.0000,1.3000,0.3000,0.5210,0.4000,0.3570,iguodan01
9,2014,NBA,Kawhi Leonard,22,SAS,5,33.4000,17.8000,6.4000,2.0000,1.6000,1.2000,0.6120,0.5790,0.7830,leonaka01


# Stage 2.) Compile TEAM related dataframes from yearly source files   

## Regular Season Team Stats

### BASIC TEAM STATS IN REGULAR SEASON. Compile yearly data into one dataframe
 <a class="anchor" id="Step11"></a>

#### Create dataframes of yearly data

In [117]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Regular Season\\NBA Basic Team (TEAM) Stats',0)

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1.0000,Dallas Mavericks*,82,241.5000,40.5000,88.2000,0.4590,6.2000,17.8000,0.3480,34.3000,70.4000,0.4880,18.0000,22.6000,0.7960,14.3000,31.0000,45.3000,23.9000,8.0000,5.3000,12.2000,19.6000,105.2000,2004


###### Validate that all of the yearly data files contain different data

In [118]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [119]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [120]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_basic_stat_in_regular_season_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [121]:
validate_completeness_of_data(nba_team_basic_stat_in_regular_season_df,2004,2024)

###### Clean team names

In [122]:
clean_column_values(nba_team_basic_stat_in_regular_season_df,'Team')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


### OPPONENT BASIC STATS AGAINST TEAM IN REGULAR SEASON. Compile yearly data into one dataframe
 <a class="anchor" id="Step12"></a>

#### Create dataframes of yearly data

In [123]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Regular Season\\NBA Basic Team (OPPONENT) Stats',0)

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1.0000,San Antonio Spurs*,82,240.9000,31.9000,77.9000,0.4090,3.8000,11.6000,0.3270,28.1000,66.3000,0.4230,16.7000,22.5000,0.7440,11.1000,30.0000,41.1000,17.3000,7.7000,4.6000,15.3000,22.4000,84.3000,2004


###### Validate that all of the yearly data files contain different data

In [124]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [125]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [126]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_basic_opponent_stat_in_regular_season_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [127]:
validate_completeness_of_data(nba_team_basic_opponent_stat_in_regular_season_df,2004,2024)

###### Clean team names

In [128]:
clean_column_values(nba_team_basic_opponent_stat_in_regular_season_df,'Team')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


### ADVANCED TEAM STATS IN REGULAR SEASON. Compile yearly data into one dataframe
 <a class="anchor" id="Step13"></a>

#### Create dataframes of yearly data

In [129]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Regular Season\\NBA Advanced Team Stats',1)

,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,Unnamed: 17,eFG%,TOV%,ORB%,FT/FGA,Unnamed: 22,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Unnamed: 27,Arena,Attend.,Attend./G,Year
0,1.0000,San Antonio Spurs*,27.3000,57.0000,25.0000,62,20,7.2200,0.2900,7.5100,102.2000,94.1000,8.1000,89.2000,0.3220,0.1770,0.5110,NaN,0.4730,14.1000,29.5000,0.2190,NaN,0.4330,14.8000,74.6000,0.2150,NaN,SBC Center,739706,18042,2004


###### Validate that all of the yearly data files contain different data

In [130]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [131]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [132]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_advanced_team_stats',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [133]:
validate_completeness_of_data(nba_advanced_team_stats,2004,2024)

###### Clean team names

In [134]:
clean_column_values(nba_advanced_team_stats,'Team')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


### TEAM SHOOTING IN REGULAR SEASON. Compile yearly data into one dataframe
 <a class="anchor" id="Step14"></a>

#### Create dataframes of yearly data

In [135]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Regular Season\\NBA Shooting Team (TEAM) Stats',1)

,Rk,Team,G,MP,FG%,Dist.,Unnamed: 6,2P,0-3,3-10,10-16,16-3P,3P,Unnamed: 13,2P.1,0-3.1,3-10.1,10-16.1,16-3P.1,3P.1,Unnamed: 20,2P.2,3P.2,Unnamed: 23,%FGA,Md.,Unnamed: 26,%FGA.1,Md..1,Unnamed: 29,%3PA,3P%,Unnamed: 32,Att.,Md..2,Year
0,1.0000,Atlanta Hawks,82,19905,0.4330,11.7000,NaN,0.8090,0.3030,0.1480,0.1530,0.2050,0.1910,NaN,0.4560,0.5770,0.3710,0.3630,0.4090,0.3350,NaN,0.5370,0.8450,NaN,0.0450,289,NaN,0.2320,767,NaN,0.2920,0.3700,NaN,2.0000,0.0000,2004


###### Validate that all of the yearly data files contain different data

In [136]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [137]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [138]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_shooting_stat_in_regular_season_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [139]:
validate_completeness_of_data(nba_team_shooting_stat_in_regular_season_df,2004,2024)

###### Clean team names

In [140]:
clean_column_values(nba_team_shooting_stat_in_regular_season_df,'Team')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


### OPPONENT SHOOTING AGAINST TEAM IN REGULAR SEASON. Compile yearly data into one dataframe
 <a class="anchor" id="Step15"></a>

#### Create dataframes of yearly data

In [141]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Regular Season\\NBA Shooting Team (OPPONENT) Stats',1)

,Rk,Team,G,MP,FG%,Dist.,Unnamed: 6,2P,0-3,3-10,10-16,16-3P,3P,Unnamed: 13,2P.1,0-3.1,3-10.1,10-16.1,16-3P.1,3P.1,Unnamed: 20,2P.2,3P.2,Unnamed: 23,%FGA,Md.,Unnamed: 26,%FGA.1,Md..1,Unnamed: 29,%3PA,3P%,Year
0,1.0000,Atlanta Hawks,82,19905,0.4400,11.5000,NaN,0.8000,0.3120,0.1610,0.1410,0.1870,0.2000,NaN,0.4610,0.5750,0.3860,0.3530,0.4160,0.3580,NaN,0.5530,0.8860,NaN,0.0480,294,NaN,0.2400,840,NaN,0.2610,0.3960,2004


###### Validate that all of the yearly data files contain different data

In [142]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [143]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [144]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_opponent_shooting_stat_in_regular_season_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [145]:
validate_completeness_of_data(nba_team_opponent_shooting_stat_in_regular_season_df,2004,2024)

###### Clean team names

In [146]:
clean_column_values(nba_team_opponent_shooting_stat_in_regular_season_df,'Team')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


### Regular Season Team Standings
 <a class="anchor" id="Step22"></a>

#### Create dataframes of yearly data. Remove columns causing load issues (Division data and Month Data) that we're not interested in anyway

In [147]:
folder_path = f'{source_data_path}\\NBA Standings'

master_df_list = []

cumulative_row_counter = 0

for file_name in os.listdir(folder_path):

    if file_name == 'desktop.ini':

        continue

    # Assign season value
    year = file_name.split(' ')[0]

    # Create dataframe name
    dataframe_name = f'_{file_name.split(".")[0].lower().replace(" ","_")}_df'

    # Store dataframe globally with specified name
    globals()[dataframe_name] = pd.read_csv(f'{folder_path}\\{file_name}',header=1)

    # Add a Year Column and assign to year
    globals()[dataframe_name]['Year'] = int(year)
    
    ## Remove columns causing load issues we don't need anyway ##
    list_to_remove = ['M','E','W','A','C','SE','NW','P','SW','Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep']
    
    for column in list_to_remove:
        if column in list(globals()[dataframe_name].columns):
            globals()[dataframe_name].drop(column,axis=1,inplace=True)

    # Append to master_df_list
    master_df_list.append(globals()[dataframe_name]) 

    # Cumulative_row_counter
    cumulative_row_counter = cumulative_row_counter + len(globals()[dataframe_name])

master_df_list[0].head(1)

,Rk,Team,Overall,Home,Road,Pre,Post,≤3,≥10,Year
0,1,Indiana Pacers,61-21,34-7,27-14,39-14,22-7,7-6,30-8,2004


###### Validate that all of the yearly data files contain different data

In [148]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [149]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [150]:
concat_dataframes_and_validate_and_reset_index_and_sort('regular_season_standing_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [151]:
validate_completeness_of_data(regular_season_standing_df,2004,2024)

###### Clean team names

In [152]:
clean_column_values(regular_season_standing_df,'Team')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


## Team Playoff Stats

### BASIC TEAM STATS IN PLAYOFFS. Compile yearly data into one dataframe
 <a class="anchor" id="Step16"></a>

#### Create dataframes of yearly data

In [153]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Playoffs\\Team Basic Stats',0)

,Rk,Tm,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1.0000,Dallas Mavericks,5,240.0000,38.2000,96.4000,0.3960,5.0000,17.4000,0.2870,33.2000,79.0000,0.4200,18.2000,23.6000,0.7710,20.4000,31.8000,52.2000,20.6000,9.2000,6.6000,13.6000,21.6000,99.6000,2004


###### Validate that all of the yearly data files contain different data

In [154]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [155]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [156]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_basic_stat_in_playoffs_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [157]:
validate_completeness_of_data(nba_team_basic_stat_in_playoffs_df,2004,2024)

###### Clean team names

In [158]:
clean_column_values(nba_team_basic_stat_in_playoffs_df,'Tm')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


### OPPONENT BASIC STATS AGAINST TEAM IN PLAYOFFS. Compile yearly data into one dataframe
 <a class="anchor" id="Step17"></a>

#### Create dataframes of yearly data

In [159]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Playoffs\\Basic Opponent Stats Against Team',0)

,Rk,Tm,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1.0000,Detroit Pistons,23,244.3000,29.5000,75.2000,0.3920,4.7000,17.0000,0.2770,24.8000,58.2000,0.4260,17.0000,23.7000,0.7210,10.9000,29.7000,40.6000,18.6000,8.0000,5.1000,15.6000,26.0000,80.7000,2004


###### Validate that all of the yearly data files contain different data

In [160]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [161]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [162]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_basic_opponent_stat_in_playoffs_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [163]:
validate_completeness_of_data(nba_team_basic_opponent_stat_in_playoffs_df,2004,2024)

###### Clean team names

In [164]:
clean_column_values(nba_team_basic_opponent_stat_in_playoffs_df,'Tm')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [165]:
validate_dataframes_using_column_values(nba_team_basic_opponent_stat_in_playoffs_df,'Tm',nba_team_basic_stat_in_playoffs_df,'Tm')

### ADVANCED TEAM STATS IN PLAYOFFS. Compile yearly data into one dataframe

 <a class="anchor" id="Step18"></a>

#### Create dataframes of yearly data

In [166]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Playoffs\\Advanced',1)

,Rk,Tm,Age,W,L,W/L%,PW,PL,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,Unnamed: 15,eFG%,TOV%,ORB%,FT/FGA,Unnamed: 20,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Year
0,1.0000,Detroit Pistons,27.3000,16.0000,7.0000,0.6960,17,6,99.3000,92.0000,7.3000,86.1000,0.3590,0.1820,0.4930,NaN,0.4390,14.4000,30.2000,0.2630,NaN,0.4230,15.4000,74.6000,0.2270,2004


###### Validate that all of the yearly data files contain different data

In [167]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [168]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [169]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_advanced_stat_in_playoffs_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [170]:
validate_completeness_of_data(nba_team_advanced_stat_in_playoffs_df,2004,2024)

###### Clean team names

In [171]:
clean_column_values(nba_team_advanced_stat_in_playoffs_df,'Tm')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [172]:
validate_dataframes_using_column_values(nba_team_advanced_stat_in_playoffs_df,'Tm',nba_team_basic_stat_in_playoffs_df,'Tm')

### TEAM SHOOTING IN PLAYOFFS. Compile yearly data into one dataframe
 <a class="anchor" id="Step19"></a>

#### Create dataframes of yearly data

In [173]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Playoffs\\Team Shooting',1)

,Rk,Tm,G,MP,FG%,Dist.,Unnamed: 6,2P,0-3,3-10,10-16,16-3P,3P,Unnamed: 13,2P.1,0-3.1,3-10.1,10-16.1,16-3P.1,3P.1,Unnamed: 20,2P.2,3P.2,Unnamed: 23,%FGA,Md.,Unnamed: 26,%3PA,3P%,Unnamed: 29,Att.,#,Year
0,1.0000,Boston Celtics,4,960,0.4080,12.7000,NaN,0.7770,0.3230,0.1100,0.1310,0.2130,0.2230,NaN,0.4290,0.5270,0.3550,0.3780,0.3500,0.3330,NaN,0.5530,0.8570,NaN,0.0460,12,NaN,0.2060,0.5380,NaN,1.0000,0.0000,2004


###### Validate that all of the yearly data files contain different data

In [174]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [175]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [176]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_shooting_in_playoffs_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [177]:
validate_completeness_of_data(nba_team_shooting_in_playoffs_df,2004,2024)

###### Clean team names

In [178]:
clean_column_values(nba_team_shooting_in_playoffs_df,'Tm')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [179]:
validate_dataframes_using_column_values(nba_team_shooting_in_playoffs_df,'Tm',nba_team_basic_stat_in_playoffs_df,'Tm')

### OPPONENT SHOOTING AGAINST TEAM IN PLAYOFFS. Compile yearly data into one dataframe
 <a class="anchor" id="Step20"></a>

#### Create dataframes of yearly data

In [180]:
create_list_of_dataframes_for_yearly_stats(f'{source_data_path}\\Team Stats\\Playoffs\\Opponent Shooting Stats Against Team',1)

,Rk,Tm,G,MP,FG%,Dist.,Unnamed: 6,2P,0-3,3-10,10-16,16-3P,3P,Unnamed: 13,2P.1,0-3.1,3-10.1,10-16.1,16-3P.1,3P.1,Unnamed: 20,2P.2,3P.2,Unnamed: 23,%FGA,Md.,Unnamed: 26,%3PA,3P%,Year
0,1.0000,Boston Celtics,4,960,0.4590,12.5000,NaN,0.7810,0.3220,0.1070,0.1300,0.2220,0.2190,NaN,0.4770,0.6510,0.2500,0.4090,0.3730,0.3920,NaN,0.5320,0.8620,NaN,0.0590,20,NaN,0.2030,0.4670,2004


###### Validate that all of the yearly data files contain different data

In [181]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [182]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [183]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_opponent_shooting_stat_in_playoffs_df',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [184]:
validate_completeness_of_data(nba_team_opponent_shooting_stat_in_playoffs_df,2004,2024)

###### Clean team names

In [185]:
clean_column_values(nba_team_opponent_shooting_stat_in_playoffs_df,'Tm')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [186]:
validate_dataframes_using_column_values(nba_team_opponent_shooting_stat_in_playoffs_df,'Tm',nba_team_basic_stat_in_playoffs_df,'Tm')

### TEAM PLAYOFF STANDINGS. Compile yearly data into onedataframe 
 <a class="anchor" id="Step21"></a>

#### Create dataframes of yearly data

In [187]:
folder_path = f'{source_data_path}\\NBA Playoff Results\\Playoff Standings'
    
master_df_list = []

cumulative_row_counter = 0

for file_name in os.listdir(folder_path):

    if file_name == 'desktop.ini':

        continue

    # Assign season value
    year = file_name.split(' ')[0]

    # Create dataframe name
    dataframe_name = f'_{file_name.split(".")[0].lower().replace(" ","_")}_df'

    # Store dataframe globally with specified name
    globals()[dataframe_name] = pd.read_csv(f'{folder_path}\\{file_name}',header=1)

    # Add a Year Column and assign to year
    globals()[dataframe_name]['Year'] = int(year)

    # Remove division columns from dataframe
    list_of_columns_to_drop = ['A','C','M','P','SW','SE','NW','Apr','May','Jun','Jul','Aug','Sep','Oct','≤3','≥10']
        
    for column in list_of_columns_to_drop:
        
        if column in list(globals()[dataframe_name].columns):
        
            globals()[dataframe_name] = globals()[dataframe_name].drop(column,axis=1)
    
    # Append to master_df_list
    master_df_list.append(globals()[dataframe_name]) 

    # Cumulative_row_counter
    cumulative_row_counter = cumulative_row_counter + len(globals()[dataframe_name])


master_df_list[0].head(1)

,Rk,Team,Overall,Home,Road,E,W,Year
0,1,Detroit Pistons,16-7,10-3,6-4,12-6,4-1,2004


###### Validate that all of the yearly data files contain different data

In [188]:
check_all_unique_dataframes(master_df_list,'Year')

###### Validate that the column structure for every data frame is the same.

In [189]:
validate_column_structure_is_same(master_df_list)

###### Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes

In [190]:
concat_dataframes_and_validate_and_reset_index_and_sort('nba_team_playoff_standing',master_df_list,cumulative_row_counter)

###### Validate we have all of the data we need from 2004-2023

In [191]:
validate_completeness_of_data(nba_team_playoff_standing,2004,2024)

###### Clean team names

In [192]:
clean_column_values(nba_team_playoff_standing,'Team')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


###### Validate against dataframe

In [193]:
validate_dataframes_using_column_values(nba_team_playoff_standing,'Team',nba_team_basic_stat_in_playoffs_df,'Tm')

# Stage 3.) Standardize primary keys across player tables and team tables

## 3A. Create standardized "Player Unique ID" column for regular season stats, playoff season stats, and all awards. There are a few different names for this field
 <a class="anchor" id="Step23"></a>

In [194]:
list_of_dataframes_to_rename_column = [basic_player_stats_regular_season_df,
                                       advanced_player_stats_in_regular_season_df,
                                       player_shooting_in_regular_season_df,
                                       player_play_by_play_in_regular_season_df,
                                       player_basic_playoff_stats_df,
                                       player_advanced_stats_in_playoffs_df,
                                       player_shooting_stats_in_playoffs_df,
                                       player_play_by_play_stats_in_playoffs_df,
                                       mvp_vote_df,
                                       roy_vote_df,
                                       all_nba_df,
                                       all_defense_df,
                                       mip_df,
                                       six_man_df,
                                       dpoy_df,
                                       finals_mvp]

list_of_player_name_columns_observed = ['Player-additional',
                                        '-additional',
                                        '-9999']

for i, dataframe_to_rename_column in enumerate(list_of_dataframes_to_rename_column):
    
    column_name_to_change = 'RAISE_ERROR_IF_I_DONT_CHANGE'
    
    # Check to see which it is 
    for observed_name in list_of_player_name_columns_observed:
        
        if observed_name in list(dataframe_to_rename_column.columns):
            
            column_name_to_change = observed_name
    
    # Rename it
    dataframe_to_rename_column = dataframe_to_rename_column.rename({column_name_to_change:'player_id'},axis=1)
    
    # Assign to list
    list_of_dataframes_to_rename_column[i] = dataframe_to_rename_column
    
# Assign changes from list to actual dataframes
basic_player_stats_regular_season_df,advanced_player_stats_in_regular_season_df,player_shooting_in_regular_season_df,player_play_by_play_in_regular_season_df,player_basic_playoff_stats_df,player_advanced_stats_in_playoffs_df,player_shooting_stats_in_playoffs_df,player_play_by_play_stats_in_playoffs_df,mvp_vote_df,roy_vote_df,all_nba_df,all_defense_df,mip_df,six_man_df,dpoy_df,finals_mvp = list_of_dataframes_to_rename_column 

### Create standardized "Team Name" column for ALL dataframes. There are team column names of "Tm" and "Team". Convert "Tm" to "Team" for all dataframes with "Tm"
 <a class="anchor" id="Step24"></a>

In [195]:
list_of_all_dataframes = [basic_player_stats_regular_season_df,
                           advanced_player_stats_in_regular_season_df,
                           player_shooting_in_regular_season_df,
                           player_play_by_play_in_regular_season_df,
                           player_basic_playoff_stats_df,
                           player_advanced_stats_in_playoffs_df,
                           player_shooting_stats_in_playoffs_df,
                           player_play_by_play_stats_in_playoffs_df,
                           mvp_vote_df,
                           roy_vote_df,
                           all_nba_df,
                           all_defense_df,
                           mip_df,
                           six_man_df,
                           dpoy_df,
                           finals_mvp,
                          nba_team_basic_stat_in_regular_season_df,
                          nba_team_basic_opponent_stat_in_regular_season_df,
                          nba_advanced_team_stats,
                          nba_team_shooting_stat_in_regular_season_df,
                          nba_team_opponent_shooting_stat_in_regular_season_df,
                          nba_team_basic_stat_in_playoffs_df,
                          nba_team_basic_opponent_stat_in_playoffs_df,
                          nba_team_advanced_stat_in_playoffs_df,
                          nba_team_shooting_in_playoffs_df,
                          nba_team_opponent_shooting_stat_in_playoffs_df,
                          nba_team_playoff_standing,
                          regular_season_standing_df
                         ]

list_of_player_name_columns_observed = ['Tm','Team']

for i, dataframe_to_rename_column in enumerate(list_of_all_dataframes):
        
    break_flag = 0
    
    # Check to see which it is 
    for observed_name in list_of_player_name_columns_observed:
        
        if observed_name in list(dataframe_to_rename_column.columns):
            
            break_flag = break_flag + 1
            
            column_name_to_change = observed_name
    
    if break_flag!=1:
        print(i)
        break
        
    
    # Rename it
    dataframe_to_rename_column = dataframe_to_rename_column.rename({column_name_to_change:'Team'},axis=1)
    
    # Assign to list
    list_of_all_dataframes[i] = dataframe_to_rename_column
    
# Assign changes from list to actual dataframes
basic_player_stats_regular_season_df,advanced_player_stats_in_regular_season_df,player_shooting_in_regular_season_df,player_play_by_play_in_regular_season_df,player_basic_playoff_stats_df,player_advanced_stats_in_playoffs_df,player_shooting_stats_in_playoffs_df,player_play_by_play_stats_in_playoffs_df,mvp_vote_df,roy_vote_df,all_nba_df,all_defense_df,mip_df,six_man_df,dpoy_df,finals_mvp,nba_team_basic_stat_in_regular_season_df,nba_team_basic_opponent_stat_in_regular_season_df,nba_advanced_team_stats,nba_team_shooting_stat_in_regular_season_df,nba_team_opponent_shooting_stat_in_regular_season_df,nba_team_basic_stat_in_playoffs_df,nba_team_basic_opponent_stat_in_playoffs_df,nba_team_advanced_stat_in_playoffs_df,nba_team_shooting_in_playoffs_df,nba_team_opponent_shooting_stat_in_playoffs_df,nba_team_playoff_standing,regular_season_standing_df = list_of_all_dataframes 

# 3B. Rename columns to be desired field name in database. Also use this step to remove redundant columns and decided what final columns will be

## REGULAR SEASON PLAYER STATS

### Regular Season Basic Player Stats

In [196]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
"GS":f"{unique_table_name}Games Started",
"MP":f"{unique_table_name}Minutes Played Per Game",
"FG":f"{unique_table_name}Field Goals Made Per Game",
"FGA":f"{unique_table_name}Field Goals Attempted Per Game",
"FG%":f"{unique_table_name}Average Field Goal % For Season",
"3P":f"{unique_table_name}3 Pointers Made Per Game",
"3PA":f"{unique_table_name}3 Point Attempts Per Game",
"3P%":f"{unique_table_name}Average 3 Point % For Season",
"2P":f"{unique_table_name}2 Pointers Made Per Game",
"2PA":f"{unique_table_name}2 Point Attempts Per Game",
"2P%":f"{unique_table_name}Average 2 Point % For Season",
"eFG%":f"{unique_table_name}Average Effective Field Goal % For Season",
"FT":f"{unique_table_name}Free Throws Made Per Game",
"FTA":f"{unique_table_name}Free Throw Attempts Per Game",
"FT%":f"{unique_table_name}Average Free Throw % For Season",
"ORB":f"{unique_table_name}Offense Rebounds Per Game",
"DRB":f"{unique_table_name}Defense Rebounds Per Game",
"TRB":f"{unique_table_name}Total Rebounds Per Game",
"AST":f"{unique_table_name}Assists Per Game",
"STL":f"{unique_table_name}Steals Per Game",
"BLK":f"{unique_table_name}Blocks Per Game",
"TOV":f"{unique_table_name}Turnovers Per Game",
"PF":f"{unique_table_name}Personal Fouls Per Game",
"PTS":f"{unique_table_name}Points Per Game",
"Year":"Season",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
basic_player_stats_regular_season_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Rank']

for item in list_of_columns_to_remove:
    basic_player_stats_regular_season_df = basic_player_stats_regular_season_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in basic_player_stats_regular_season_df.columns:
    
    print(f"""
    {column}""")

basic_player_stats_regular_season_df.head(1)


    Player

    Position

    Age

    Team

    Games Played

    Games Started

    Minutes Played Per Game

    Field Goals Made Per Game

    Field Goals Attempted Per Game

    Average Field Goal % For Season

    3 Pointers Made Per Game

    3 Point Attempts Per Game

    Average 3 Point % For Season

    2 Pointers Made Per Game

    2 Point Attempts Per Game

    Average 2 Point % For Season

    Average Effective Field Goal % For Season

    Free Throws Made Per Game

    Free Throw Attempts Per Game

    Average Free Throw % For Season

    Offense Rebounds Per Game

    Defense Rebounds Per Game

    Total Rebounds Per Game

    Assists Per Game

    Steals Per Game

    Blocks Per Game

    Turnovers Per Game

    Personal Fouls Per Game

    Points Per Game

    player_id

    Season


,Player,Position,Age,Team,Games Played,Games Started,Minutes Played Per Game,Field Goals Made Per Game,Field Goals Attempted Per Game,Average Field Goal % For Season,3 Pointers Made Per Game,3 Point Attempts Per Game,Average 3 Point % For Season,2 Pointers Made Per Game,2 Point Attempts Per Game,Average 2 Point % For Season,Average Effective Field Goal % For Season,Free Throws Made Per Game,Free Throw Attempts Per Game,Average Free Throw % For Season,Offense Rebounds Per Game,Defense Rebounds Per Game,Total Rebounds Per Game,Assists Per Game,Steals Per Game,Blocks Per Game,Turnovers Per Game,Personal Fouls Per Game,Points Per Game,player_id,Season
0,Ivica Zubac,C,26,LAC,54,54,26.4000,4.9000,7.5000,0.6530,0.0000,0.0000,NaN,4.9000,7.5000,0.6530,0.6530,1.7000,2.5000,0.6870,2.9000,6.2000,9.2000,1.4000,0.2000,1.2000,1.2000,2.7000,11.5000,zubaciv01,2024


### Regular Season Advanced Player Stats

In [197]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
"GS":f"{unique_table_name}Games Started",
'PER':f'{unique_table_name}Player Efficiency',
'DBPM':f'{unique_table_name}Defensive Box Plus/Minus',
'VORP':f'{unique_table_name}Value Above Replacement Player',
'TRB%':f'{unique_table_name}Total Rebounding %',
'ORB%':f'{unique_table_name}Offensive Rebounding %',
'WS':f'{unique_table_name}Win Shares',
'BLK%':f'{unique_table_name}Block %',
'DWS':f'{unique_table_name}Total Defensive Win Shares For Season',
'TOV%':f'{unique_table_name}Turnover % for Season',
'STL%':f'{unique_table_name}Steal Percentage for Season',
'FTr':f'{unique_table_name}Free Throw Rate for Season',
'DRB%':f'{unique_table_name}Defensive Rebound Percentage',
'3PAr':f'{unique_table_name}3 Point Attempt Rate',
'AST%':f'{unique_table_name}Assist % For Season',
'OBPM':f'{unique_table_name}Offensive Box Plus/Minus For Season',
'BPM':f'{unique_table_name}Box Plus/Minus For Season',
'WS/48':f'{unique_table_name}Win Shares Per 48 Minutes Played For the Season',
'USG%':f'{unique_table_name}Usage % For Season',
'OWS':f'{unique_table_name}Total Offensive Win Shares For Season',
'TS%':f'{unique_table_name}True Shooting % For Season',
'MP':f'{unique_table_name}Total Minutes Played During Season',
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name",}

# Replace names in dataframe
advanced_player_stats_in_regular_season_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 19','Unnamed: 24','Rank']

for item in list_of_columns_to_remove:
    advanced_player_stats_in_regular_season_df = advanced_player_stats_in_regular_season_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in advanced_player_stats_in_regular_season_df.columns:
    
    print(f"""
    {column}""")

advanced_player_stats_in_regular_season_df.head(1)


    Player

    Position

    Age

    Team

    Games Played

    Total Minutes Played During Season

    Player Efficiency

    True Shooting % For Season

    3 Point Attempt Rate

    Free Throw Rate for Season

    Offensive Rebounding %

    Defensive Rebound Percentage

    Total Rebounding %

    Assist % For Season

    Steal Percentage for Season

    Block %

    Turnover % for Season

    Usage % For Season

    Total Offensive Win Shares For Season

    Total Defensive Win Shares For Season

    Win Shares

    Win Shares Per 48 Minutes Played For the Season

    Offensive Box Plus/Minus For Season

    Defensive Box Plus/Minus

    Box Plus/Minus For Season

    Value Above Replacement Player

    player_id

    Year


,Player,Position,Age,Team,Games Played,Total Minutes Played During Season,Player Efficiency,True Shooting % For Season,3 Point Attempt Rate,Free Throw Rate for Season,Offensive Rebounding %,Defensive Rebound Percentage,Total Rebounding %,Assist % For Season,Steal Percentage for Season,Block %,Turnover % for Season,Usage % For Season,Total Offensive Win Shares For Season,Total Defensive Win Shares For Season,Win Shares,Win Shares Per 48 Minutes Played For the Season,Offensive Box Plus/Minus For Season,Defensive Box Plus/Minus,Box Plus/Minus For Season,Value Above Replacement Player,player_id,Year
0,Ivica Zubac,C,26,LAC,54,1428,18.6000,0.6700,0.0000,0.3320,13.1000,25.9000,19.7000,7.4000,0.4000,4.0000,12.3000,16.2000,3.4000,1.8000,5.2000,0.1750,0.5000,-0.2000,0.3000,0.9000,zubaciv01,2024


### Regular Season Shooting Player Stats

In [198]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
'MP':f'{unique_table_name}Total Minutes Played During Season',
'FG%':f'{unique_table_name}Field Goal %',
'Dist.':f'{unique_table_name}Average distance of shot',                         
'2P':f'{unique_table_name}2 Pointer Attempt Rate For Season as % of total field goals attempted',
'0-3':f'{unique_table_name}% of FGA between 0-3 ft.',
'3-10':f'{unique_table_name}% of FGA between 3-10 ft.',                         
'10-16':f'{unique_table_name}% of FGA between 10-16 ft.',
'16-3P':f'{unique_table_name}% of FGA between 16-3P ft.',
'3P':f'{unique_table_name}3 Pointer Attempt Rate For Season as % of total field goals attempted',
'2P.1':f'{unique_table_name}FG % of 2 Point Attempts',
'0-3.1':f'{unique_table_name}FG % of Attempts between 0-3 ft.',
'3-10.1':f'{unique_table_name}FG % of Attempts between 3-10 ft.',                         
'10-16.1':f'{unique_table_name}FG % of Attempts between 10-16 ft.',
'16-3P.1':f'{unique_table_name}FG % of Attempts between 16-3P ft.',
'3P.1':f'{unique_table_name}FG % of 3 Point Attempts',                       
'2P.2':f'{unique_table_name}% of Made 2 Point Fields Goals That Were Assisted',                         
'3P.2':f'{unique_table_name}% of Made 3 Point Fields Goals That Were Assisted',
'%FGA':f'{unique_table_name}% of Field Goal attempts that are dunk attempts',
'#':f'{unique_table_name}Number of made dunk attempts',
'%3PA':f'{unique_table_name}% of 3 Point Attempts that come from the corner',
'3P%':f'{unique_table_name}% of 3 Point corner attempts that are made',
'Att.':f'{unique_table_name}Heave Attempts',
'#.1':f'{unique_table_name}Number of made heaves',                     
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
player_shooting_in_regular_season_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 9','Unnamed: 16','Unnamed: 23','Unnamed: 26','Unnamed: 29','Unnamed: 32','Rank']

for item in list_of_columns_to_remove:
    player_shooting_in_regular_season_df = player_shooting_in_regular_season_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in player_shooting_in_regular_season_df.columns:
    
    print(f"""
    {column}""")

player_shooting_in_regular_season_df.head(1)


    Player

    Position

    Age

    Team

    Games Played

    Total Minutes Played During Season

    Field Goal %

    Average distance of shot

    2 Pointer Attempt Rate For Season as % of total field goals attempted

    % of FGA between 0-3 ft.

    % of FGA between 3-10 ft.

    % of FGA between 10-16 ft.

    % of FGA between 16-3P ft.

    3 Pointer Attempt Rate For Season as % of total field goals attempted

    FG % of 2 Point Attempts

    FG % of Attempts between 0-3 ft.

    FG % of Attempts between 3-10 ft.

    FG % of Attempts between 10-16 ft.

    FG % of Attempts between 16-3P ft.

    FG % of 3 Point Attempts

    % of Made 2 Point Fields Goals That Were Assisted

    % of Made 3 Point Fields Goals That Were Assisted

    % of Field Goal attempts that are dunk attempts

    Number of made dunk attempts

    % of 3 Point Attempts that come from the corner

    % of 3 Point corner attempts that are made

    Heave Attempts

    Number of made heaves

    player_

,Player,Position,Age,Team,Games Played,Total Minutes Played During Season,Field Goal %,Average distance of shot,2 Pointer Attempt Rate For Season as % of total field goals attempted,% of FGA between 0-3 ft.,% of FGA between 3-10 ft.,% of FGA between 10-16 ft.,% of FGA between 16-3P ft.,3 Pointer Attempt Rate For Season as % of total field goals attempted,FG % of 2 Point Attempts,FG % of Attempts between 0-3 ft.,FG % of Attempts between 3-10 ft.,FG % of Attempts between 10-16 ft.,FG % of Attempts between 16-3P ft.,FG % of 3 Point Attempts,% of Made 2 Point Fields Goals That Were Assisted,% of Made 3 Point Fields Goals That Were Assisted,% of Field Goal attempts that are dunk attempts,Number of made dunk attempts,% of 3 Point Attempts that come from the corner,% of 3 Point corner attempts that are made,Heave Attempts,Number of made heaves,player_id,Year
0,Ivica Zubac,C,26,LAC,54,1428,0.6530,3.6000,1.0000,0.5220,0.4430,0.0350,0.0000,0.0000,0.6530,0.6870,0.6200,0.5710,NaN,NaN,0.7650,NaN,0.2650,92,NaN,NaN,0,0,zubaciv01,2024


### Regular Season Play by Play Stats

In [199]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
'MP':f'{unique_table_name}Total Minutes Played During Season',
                         
'PG%':f'{unique_table_name}Percentage of Total Minutes Played at Point Guard',                         
'SG%':f'{unique_table_name}Percentage of Total Minutes Played at Shooting Guard',    
'SF%':f'{unique_table_name}Percentage of Total Minutes Played at Small Forward',    
'PF%':f'{unique_table_name}Percentage of Total Minutes Played at Power Forward',    
'C%':f'{unique_table_name}Percentage of Total Minutes Played at Center',    
'OnCourt':f'{unique_table_name}Plus/Minus Per 100 Possessions (On Court)',    
'On-Off':f'{unique_table_name}Plus/Minus Net Per 100 Possessions (On/off)',    
'BadPass':f'{unique_table_name}Total Turnovers by Bad Pass',    
'LostBall':f'{unique_table_name}Total Lost Ball Turnovers',    
'Shoot':f'{unique_table_name}Shooting Fouls Committed',    
'Off.':f'{unique_table_name}Offensive Fouls Committed', 
'Shoot.1':f'{unique_table_name}Shooting Fouls Drawn',    
'Off..1':f'{unique_table_name}Offensive Fouls Drawn',                     
'PGA':f'{unique_table_name}Points Generated by Assists',    
'And1':f'{unique_table_name}Number of And1 Attempts After Made Field Goal',    
'Blkd':f'{unique_table_name}Field Goal Attempts Blocked',                                                      
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
player_play_by_play_in_regular_season_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Rank']

for item in list_of_columns_to_remove:
    player_play_by_play_in_regular_season_df = player_play_by_play_in_regular_season_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in player_play_by_play_in_regular_season_df.columns:
    
    print(f"""
    {column}""")

player_play_by_play_in_regular_season_df.head(1)


    Player

    Position

    Age

    Team

    Games Played

    Total Minutes Played During Season

    Percentage of Total Minutes Played at Point Guard

    Percentage of Total Minutes Played at Shooting Guard

    Percentage of Total Minutes Played at Small Forward

    Percentage of Total Minutes Played at Power Forward

    Percentage of Total Minutes Played at Center

    Plus/Minus Per 100 Possessions (On Court)

    Plus/Minus Net Per 100 Possessions (On/off)

    Total Turnovers by Bad Pass

    Total Lost Ball Turnovers

    Shooting Fouls Committed

    Offensive Fouls Committed

    Shooting Fouls Drawn

    Offensive Fouls Drawn

    Points Generated by Assists

    Number of And1 Attempts After Made Field Goal

    Field Goal Attempts Blocked

    player_id

    Year


,Player,Position,Age,Team,Games Played,Total Minutes Played During Season,Percentage of Total Minutes Played at Point Guard,Percentage of Total Minutes Played at Shooting Guard,Percentage of Total Minutes Played at Small Forward,Percentage of Total Minutes Played at Power Forward,Percentage of Total Minutes Played at Center,Plus/Minus Per 100 Possessions (On Court),Plus/Minus Net Per 100 Possessions (On/off),Total Turnovers by Bad Pass,Total Lost Ball Turnovers,Shooting Fouls Committed,Offensive Fouls Committed,Shooting Fouls Drawn,Offensive Fouls Drawn,Points Generated by Assists,Number of And1 Attempts After Made Field Goal,Field Goal Attempts Blocked,player_id,Year
0,Ivica Zubac,C,26,LAC,54,1428,NaN,NaN,NaN,NaN,100%,5.8000,2.2000,25,15,106,20,64,3.0000,183,8,33,zubaciv01,2024


## PLAYOFFS PLAYER STATS

### Playoffs Basic Player Stats

In [200]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
"GS":f"{unique_table_name}Games Started",
"MP":f"{unique_table_name}Minutes Played Per Game",
"FG":f"{unique_table_name}Field Goals Made Per Game",
"FGA":f"{unique_table_name}Field Goals Attempted Per Game",
"FG%":f"{unique_table_name}Average Field Goal % For Season",
"3P":f"{unique_table_name}3 Pointers Made Per Game",
"3PA":f"{unique_table_name}3 Point Attempts Per Game",
"3P%":f"{unique_table_name}Average 3 Point % For Season",
"2P":f"{unique_table_name}2 Pointers Made Per Game",
"2PA":f"{unique_table_name}2 Point Attempts Per Game",
"2P%":f"{unique_table_name}Average 2 Point % For Season",
"eFG%":f"{unique_table_name}Average Effective Field Goal % For Season",
"FT":f"{unique_table_name}Free Throws Made Per Game",
"FTA":f"{unique_table_name}Free Throw Attempts Per Game",
"FT%":f"{unique_table_name}Average Free Throw % For Season",
"ORB":f"{unique_table_name}Offense Rebounds Per Game",
"DRB":f"{unique_table_name}Defense Rebounds Per Game",
"TRB":f"{unique_table_name}Total Rebounds Per Game",
"AST":f"{unique_table_name}Assists Per Game",
"STL":f"{unique_table_name}Steals Per Game",
"BLK":f"{unique_table_name}Blocks Per Game",
"TOV":f"{unique_table_name}Turnovers Per Game",
"PF":f"{unique_table_name}Personal Fouls Per Game",
"PTS":f"{unique_table_name}Points Per Game",
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
player_basic_playoff_stats_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Rank']

for item in list_of_columns_to_remove:
    player_basic_playoff_stats_df = player_basic_playoff_stats_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in player_basic_playoff_stats_df.columns:
    
    print(f"""
    {column}""")

player_basic_playoff_stats_df.head(1)



    Player

    Position

    Age

    Team

    Games Played

    Games Started

    Minutes Played Per Game

    Field Goals Made Per Game

    Field Goals Attempted Per Game

    Average Field Goal % For Season

    3 Pointers Made Per Game

    3 Point Attempts Per Game

    Average 3 Point % For Season

    2 Pointers Made Per Game

    2 Point Attempts Per Game

    Average 2 Point % For Season

    Average Effective Field Goal % For Season

    Free Throws Made Per Game

    Free Throw Attempts Per Game

    Average Free Throw % For Season

    Offense Rebounds Per Game

    Defense Rebounds Per Game

    Total Rebounds Per Game

    Assists Per Game

    Steals Per Game

    Blocks Per Game

    Turnovers Per Game

    Personal Fouls Per Game

    Points Per Game

    player_id

    Year


,Player,Position,Age,Team,Games Played,Games Started,Minutes Played Per Game,Field Goals Made Per Game,Field Goals Attempted Per Game,Average Field Goal % For Season,3 Pointers Made Per Game,3 Point Attempts Per Game,Average 3 Point % For Season,2 Pointers Made Per Game,2 Point Attempts Per Game,Average 2 Point % For Season,Average Effective Field Goal % For Season,Free Throws Made Per Game,Free Throw Attempts Per Game,Average Free Throw % For Season,Offense Rebounds Per Game,Defense Rebounds Per Game,Total Rebounds Per Game,Assists Per Game,Steals Per Game,Blocks Per Game,Turnovers Per Game,Personal Fouls Per Game,Points Per Game,player_id,Year
0,Ivica Zubac,C,25,LAC,5,5,26.0000,3.4000,6.0000,0.5670,0.0000,0.0000,NaN,3.4000,6.0000,0.5670,0.5670,2.4000,3.2000,0.7500,3.2000,6.4000,9.6000,0.6000,0.6000,0.2000,2.2000,1.6000,9.2000,zubaciv01,2023


### Playoffs Advanced Player Stats

In [201]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
"GS":f"{unique_table_name}Games Started",
'PER':f'{unique_table_name}Player Efficiency',
'DBPM':f'{unique_table_name}Defensive Box Plus/Minus',
'VORP':f'{unique_table_name}Value Above Replacement Player',
'TRB%':f'{unique_table_name}Total Rebounding %',
'ORB%':f'{unique_table_name}Offensive Rebounding %',
'WS':f'{unique_table_name}Win Shares',
'BLK%':f'{unique_table_name}Block %',
'DWS':f'{unique_table_name}Total Defensive Win Shares For Season',
'TOV%':f'{unique_table_name}Turnover % for Season',
'STL%':f'{unique_table_name}Steal Percentage for Season',
'FTr':f'{unique_table_name}Free Throw Rate for Season',
'DRB%':f'{unique_table_name}Defensive Rebound Percentage',
'3PAr':f'{unique_table_name}3 Point Attempt Rate',
'AST%':f'{unique_table_name}Assist % For Season',
'OBPM':f'{unique_table_name}Offensive Box Plus/Minus For Season',
'BPM':f'{unique_table_name}Box Plus/Minus For Season',
'WS/48':f'{unique_table_name}Win Shares Per 48 Minutes Played For the Season',
'USG%':f'{unique_table_name}Usage % For Season',
'OWS':f'{unique_table_name}Total Offensive Win Shares For Season',
'TS%':f'{unique_table_name}True Shooting % For Season',
'MP':f'{unique_table_name}Total Minutes Played During Season',
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name",}

# Replace names in dataframe
player_advanced_stats_in_playoffs_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 19','Unnamed: 24','Rank']

for item in list_of_columns_to_remove:
    player_advanced_stats_in_playoffs_df = player_advanced_stats_in_playoffs_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in player_advanced_stats_in_playoffs_df.columns:
    
    print(f"""
    {column}""")

player_advanced_stats_in_playoffs_df.head(1)


    Player

    Position

    Age

    Team

    Games Played

    Total Minutes Played During Season

    Player Efficiency

    True Shooting % For Season

    3 Point Attempt Rate

    Free Throw Rate for Season

    Offensive Rebounding %

    Defensive Rebound Percentage

    Total Rebounding %

    Assist % For Season

    Steal Percentage for Season

    Block %

    Turnover % for Season

    Usage % For Season

    Total Offensive Win Shares For Season

    Total Defensive Win Shares For Season

    Win Shares

    Win Shares Per 48 Minutes Played For the Season

    Offensive Box Plus/Minus For Season

    Defensive Box Plus/Minus

    Box Plus/Minus For Season

    Value Above Replacement Player

    player_id

    Year


,Player,Position,Age,Team,Games Played,Total Minutes Played During Season,Player Efficiency,True Shooting % For Season,3 Point Attempt Rate,Free Throw Rate for Season,Offensive Rebounding %,Defensive Rebound Percentage,Total Rebounding %,Assist % For Season,Steal Percentage for Season,Block %,Turnover % for Season,Usage % For Season,Total Offensive Win Shares For Season,Total Defensive Win Shares For Season,Win Shares,Win Shares Per 48 Minutes Played For the Season,Offensive Box Plus/Minus For Season,Defensive Box Plus/Minus,Box Plus/Minus For Season,Value Above Replacement Player,player_id,Year
0,Ivica Zubac,C,25,LAC,5,130,14.1000,0.6210,0.0000,0.5330,13.4000,30.5000,21.4000,3.2000,1.1000,0.6000,22.9000,15.5000,0.1000,0.0000,0.1000,0.0410,-1.2000,-1.0000,-2.3000,0.0000,zubaciv01,2023


### Playoffs Shooting Player Stats

In [202]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
'MP':f'{unique_table_name}Total Minutes Played During Season',
'FG%':f'{unique_table_name}Field Goal %',
'Dist.':f'{unique_table_name}Average distance of shot',                         
'2P':f'{unique_table_name}2 Pointer Attempt Rate For Season as % of total field goals attempted',
'0-3':f'{unique_table_name}% of FGA between 0-3 ft.',
'3-10':f'{unique_table_name}% of FGA between 3-10 ft.',                         
'10-16':f'{unique_table_name}% of FGA between 10-16 ft.',
'16-3P':f'{unique_table_name}% of FGA between 16-3P ft.',
'3P':f'{unique_table_name}3 Pointer Attempt Rate For Season as % of total field goals attempted',
'2P.1':f'{unique_table_name}FG % of 2 Point Attempts',
'0-3.1':f'{unique_table_name}FG % of Attempts between 0-3 ft.',
'3-10.1':f'{unique_table_name}FG % of Attempts between 3-10 ft.',                         
'10-16.1':f'{unique_table_name}FG % of Attempts between 10-16 ft.',
'16-3P.1':f'{unique_table_name}FG % of Attempts between 16-3P ft.',
'3P.1':f'{unique_table_name}FG % of 3 Point Attempts',                       
'2P.2':f'{unique_table_name}% of Made 2 Point Fields Goals That Were Assisted',                         
'3P.2':f'{unique_table_name}% of Made 3 Point Fields Goals That Were Assisted',
'%FGA':f'{unique_table_name}% of Field Goal attempts that are dunk attempts',
'#':f'{unique_table_name}Number of made dunk attempts',
'%3PA':f'{unique_table_name}% of 3 Point Attempts that come from the corner',
'3P%':f'{unique_table_name}% of 3 Point corner attempts that are made',
'Att.':f'{unique_table_name}Heave Attempts',
'#.1':f'{unique_table_name}Number of made heaves',                     
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
player_shooting_stats_in_playoffs_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 9','Unnamed: 16','Unnamed: 23','Unnamed: 26','Unnamed: 29','Unnamed: 32', 'Rank']

for item in list_of_columns_to_remove:
    player_shooting_stats_in_playoffs_df = player_shooting_stats_in_playoffs_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in player_shooting_stats_in_playoffs_df.columns:
    
    print(f"""
    {column}""")

player_shooting_stats_in_playoffs_df.head(1)


    Player

    Position

    Age

    Team

    Games Played

    Total Minutes Played During Season

    Field Goal %

    Average distance of shot

    2 Pointer Attempt Rate For Season as % of total field goals attempted

    % of FGA between 0-3 ft.

    % of FGA between 3-10 ft.

    % of FGA between 10-16 ft.

    % of FGA between 16-3P ft.

    3 Pointer Attempt Rate For Season as % of total field goals attempted

    FG % of 2 Point Attempts

    FG % of Attempts between 0-3 ft.

    FG % of Attempts between 3-10 ft.

    FG % of Attempts between 10-16 ft.

    FG % of Attempts between 16-3P ft.

    FG % of 3 Point Attempts

    % of Made 2 Point Fields Goals That Were Assisted

    % of Made 3 Point Fields Goals That Were Assisted

    % of Field Goal attempts that are dunk attempts

    Number of made dunk attempts

    % of 3 Point Attempts that come from the corner

    % of 3 Point corner attempts that are made

    Heave Attempts

    Number of made heaves

    player_

,Player,Position,Age,Team,Games Played,Total Minutes Played During Season,Field Goal %,Average distance of shot,2 Pointer Attempt Rate For Season as % of total field goals attempted,% of FGA between 0-3 ft.,% of FGA between 3-10 ft.,% of FGA between 10-16 ft.,% of FGA between 16-3P ft.,3 Pointer Attempt Rate For Season as % of total field goals attempted,FG % of 2 Point Attempts,FG % of Attempts between 0-3 ft.,FG % of Attempts between 3-10 ft.,FG % of Attempts between 10-16 ft.,FG % of Attempts between 16-3P ft.,FG % of 3 Point Attempts,% of Made 2 Point Fields Goals That Were Assisted,% of Made 3 Point Fields Goals That Were Assisted,% of Field Goal attempts that are dunk attempts,Number of made dunk attempts,% of 3 Point Attempts that come from the corner,% of 3 Point corner attempts that are made,Heave Attempts,Number of made heaves,player_id,Year
0,Ivica Zubac,C,25,LAC,5,130,0.5670,3.7000,1.0000,0.4670,0.5000,0.0330,0.0000,0.0000,0.5670,0.5710,0.6000,0.0000,NaN,NaN,0.7060,NaN,0.2330,6,NaN,NaN,0,0,zubaciv01,2023


### Playoffs Play by Play Player Stats

In [203]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
'MP':f'{unique_table_name}Total Minutes Played During Season',                        
'PG%':f'{unique_table_name}Percentage of Total Minutes Played at Point Guard',                         
'SG%':f'{unique_table_name}Percentage of Total Minutes Played at Shooting Guard',    
'SF%':f'{unique_table_name}Percentage of Total Minutes Played at Small Forward',    
'PF%':f'{unique_table_name}Percentage of Total Minutes Played at Power Forward',    
'C%':f'{unique_table_name}Percentage of Total Minutes Played at Center',    
'OnCourt':f'{unique_table_name}Plus/Minus Per 100 Possessions (On Court)',    
'On-Off':f'{unique_table_name}Plus/Minus Net Per 100 Possessions (On/off)',    
'BadPass':f'{unique_table_name}Total Turnovers by Bad Pass',    
'LostBall':f'{unique_table_name}Total Lost Ball Turnovers',    
'Shoot':f'{unique_table_name}Shooting Fouls Committed',    
'Off.':f'{unique_table_name}Offensive Fouls Committed', 
'Shoot.1':f'{unique_table_name}Shooting Fouls Drawn',    
'Off..1':f'{unique_table_name}Offensive Fouls Drawn',                     
'PGA':f'{unique_table_name}Points Generated by Assists',    
'And1':f'{unique_table_name}Number of And1 Attempts After Made Field Goal',    
'Blkd':f'{unique_table_name}Field Goal Attempts Blocked',                                                      
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
player_play_by_play_stats_in_playoffs_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Rank']

for item in list_of_columns_to_remove:
    player_play_by_play_stats_in_playoffs_df = player_play_by_play_stats_in_playoffs_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in player_play_by_play_stats_in_playoffs_df.columns:
    
    print(f"""
    {column}""")

player_play_by_play_stats_in_playoffs_df.head(1)


    Player

    Position

    Age

    Team

    Games Played

    Total Minutes Played During Season

    Percentage of Total Minutes Played at Point Guard

    Percentage of Total Minutes Played at Shooting Guard

    Percentage of Total Minutes Played at Small Forward

    Percentage of Total Minutes Played at Power Forward

    Percentage of Total Minutes Played at Center

    Plus/Minus Per 100 Possessions (On Court)

    Plus/Minus Net Per 100 Possessions (On/off)

    Total Turnovers by Bad Pass

    Total Lost Ball Turnovers

    Shooting Fouls Committed

    Offensive Fouls Committed

    Shooting Fouls Drawn

    Offensive Fouls Drawn

    Points Generated by Assists

    Number of And1 Attempts After Made Field Goal

    Field Goal Attempts Blocked

    player_id

    Year


,Player,Position,Age,Team,Games Played,Total Minutes Played During Season,Percentage of Total Minutes Played at Point Guard,Percentage of Total Minutes Played at Shooting Guard,Percentage of Total Minutes Played at Small Forward,Percentage of Total Minutes Played at Power Forward,Percentage of Total Minutes Played at Center,Plus/Minus Per 100 Possessions (On Court),Plus/Minus Net Per 100 Possessions (On/off),Total Turnovers by Bad Pass,Total Lost Ball Turnovers,Shooting Fouls Committed,Offensive Fouls Committed,Shooting Fouls Drawn,Offensive Fouls Drawn,Points Generated by Assists,Number of And1 Attempts After Made Field Goal,Field Goal Attempts Blocked,player_id,Year
0,Ivica Zubac,C,25,LAC,5,130,NaN,NaN,NaN,NaN,100%,-17.6000,-21.6000,2,6,6,1,7,0.0000,8,0,4,zubaciv01,2023


## Player Awards

### MVP Award

In [204]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rank":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"First":f"{unique_table_name}First Place Votes",
"Pts Won":f"{unique_table_name}Total Voting Points Won",
"Pts Max":f"{unique_table_name}Total Voting Points Possible",
"Share":f"{unique_table_name}% of Total Voting Points Possible That Were Won",                                                                                                                  
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
mvp_vote_df.rename(name_replacement_dict,axis=1,inplace=True)

# # Get rid of dataframes as desired
list_of_columns_to_remove = ['G','PTS','TRB','AST','STL','BLK','FG%','3P%','FT%','WS','WS/48','MP','Rank']

for item in list_of_columns_to_remove:
    mvp_vote_df = mvp_vote_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in mvp_vote_df.columns:
    
    print(f"""
    {column}""")

mvp_vote_df.head(1)


    Player

    Age

    Team

    First Place Votes

    Total Voting Points Won

    Total Voting Points Possible

    % of Total Voting Points Possible That Were Won

    player_id

    Year


,Player,Age,Team,First Place Votes,Total Voting Points Won,Total Voting Points Possible,% of Total Voting Points Possible That Were Won,player_id,Year
0,Ja Morant,23,MEM,0.0000,1.0000,1000,0.0010,moranja01,2023


### ROY Award

In [205]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rank":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"First":f"{unique_table_name}First Place Votes",
"Pts Won":f"{unique_table_name}Total Voting Points Won",
"Pts Max":f"{unique_table_name}Total Voting Points Possible",
"Share":f"{unique_table_name}% of Total Voting Points Possible That Were Won",                                                                                                                  
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
roy_vote_df.rename(name_replacement_dict,axis=1,inplace=True)

# # Get rid of dataframes as desired
list_of_columns_to_remove = ['G','PTS','TRB','AST','STL','BLK','FG%','3P%','FT%','WS','WS/48','MP','Rank']

for item in list_of_columns_to_remove:
    roy_vote_df = roy_vote_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in roy_vote_df.columns:
    
    print(f"""
    {column}""")

roy_vote_df.head(1)


    Player

    Age

    Team

    First Place Votes

    Total Voting Points Won

    Total Voting Points Possible

    % of Total Voting Points Possible That Were Won

    player_id

    Year


,Player,Age,Team,First Place Votes,Total Voting Points Won,Total Voting Points Possible,% of Total Voting Points Possible That Were Won,player_id,Year
0,Jaden Ivey,20,DET,0.0000,3.0000,500,0.0060,iveyja01,2023


### All NBA Award

In [206]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rank":f"{unique_table_name}Rank",
"# Tm":f"{unique_table_name}All NBA Award (e.g. 1st Team)",
"1st Tm":f"{unique_table_name}All NBA 1st Team Votes",
"2nd Tm":f"{unique_table_name}All NBA 2nd Team Votes",
"3rd Tm":f"{unique_table_name}All NBA 3rd Team Votes",                                                  
"Pos":f"{unique_table_name}Position",                         
"Player":f"{unique_table_name}Player",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"First":f"{unique_table_name}First Place Votes",
"Pts Won":f"{unique_table_name}Total Voting Points Won",
"Pts Max":f"{unique_table_name}Total Voting Points Possible",
"Share":f"{unique_table_name}% of Total Voting Points Possible That Were Won",                                                                                                                  
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
all_nba_df.rename(name_replacement_dict,axis=1,inplace=True)

# # # Get rid of dataframes as desired
list_of_columns_to_remove = ['G', 'MP', 'PTS', 'TRB', 'AST', 'STL','BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48']

for item in list_of_columns_to_remove:
    all_nba_df = all_nba_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in all_nba_df.columns:
    
    print(f"""
    {column}""")

all_nba_df.head(1)


    All NBA Award (e.g. 1st Team)

    Position

    Player

    Age

    Team

    Total Voting Points Won

    Total Voting Points Possible

    % of Total Voting Points Possible That Were Won

    All NBA 1st Team Votes

    All NBA 2nd Team Votes

    All NBA 3rd Team Votes

    player_id

    Year


,All NBA Award (e.g. 1st Team),Position,Player,Age,Team,Total Voting Points Won,Total Voting Points Possible,% of Total Voting Points Possible That Were Won,All NBA 1st Team Votes,All NBA 2nd Team Votes,All NBA 3rd Team Votes,player_id,Year
0,ORV,G,Trae Young,24,ATL,1,500,0.0020,0.0000,0.0000,1.0000,youngtr01,2023


### All Defensive Award

In [207]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rank":f"{unique_table_name}Rank",
"# Tm":f"{unique_table_name}All NBA Award (e.g. 1st Team)",
"1st Tm":f"{unique_table_name}All NBA 1st Team Votes",
"2nd Tm":f"{unique_table_name}All NBA 2nd Team Votes",
"3rd Tm":f"{unique_table_name}All NBA 3rd Team Votes",                                                  
"Pos":f"{unique_table_name}Position",                         
"Player":f"{unique_table_name}Player",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"First":f"{unique_table_name}First Place Votes",
"Pts Won":f"{unique_table_name}Total Voting Points Won",
"Pts Max":f"{unique_table_name}Total Voting Points Possible",
"Share":f"{unique_table_name}% of Total Voting Points Possible That Were Won",                                                                                                                  
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
all_defense_df.rename(name_replacement_dict,axis=1,inplace=True)

# # # Get rid of dataframes as desired
list_of_columns_to_remove = ['G', 'MP', 'PTS', 'TRB', 'AST', 'STL','BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48','DWS','DBPM','DRtg']

for item in list_of_columns_to_remove:
    all_defense_df = all_defense_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in all_defense_df.columns:
    
    print(f"""
    {column}""")

all_defense_df.head(1)


    All NBA Award (e.g. 1st Team)

    Position

    Player

    Age

    Team

    Total Voting Points Won

    Total Voting Points Possible

    % of Total Voting Points Possible That Were Won

    player_id

    Year


,All NBA Award (e.g. 1st Team),Position,Player,Age,Team,Total Voting Points Won,Total Voting Points Possible,% of Total Voting Points Possible That Were Won,player_id,Year
0,ORV,G,Delon Wright,30,WAS,1,200,0.0050,wrighde01,2023


### Most Improved Player Award

In [208]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rank":f"{unique_table_name}Rank",
"# Tm":f"{unique_table_name}All NBA Award (e.g. 1st Team)",
"1st Tm":f"{unique_table_name}All NBA 1st Team Votes",
"2nd Tm":f"{unique_table_name}All NBA 2nd Team Votes",
"3rd Tm":f"{unique_table_name}All NBA 3rd Team Votes",                                                  
"Pos":f"{unique_table_name}Position",                         
"Player":f"{unique_table_name}Player",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"First":f"{unique_table_name}First Place Votes",
"Pts Won":f"{unique_table_name}Total Voting Points Won",
"Pts Max":f"{unique_table_name}Total Voting Points Possible",
"Share":f"{unique_table_name}% of Total Voting Points Possible That Were Won",                                                                                                                  
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
mip_df.rename(name_replacement_dict,axis=1,inplace=True)

# # Get rid of dataframes as desired
list_of_columns_to_remove = ['G', 'MP', 'PTS', 'TRB', 'AST', 'STL','BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48', 'Rank']

for item in list_of_columns_to_remove:
    mip_df = mip_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in mip_df.columns:
    
    print(f"""
    {column}""")

mip_df.head(1)


    Player

    Age

    Team

    First Place Votes

    Total Voting Points Won

    Total Voting Points Possible

    % of Total Voting Points Possible That Were Won

    player_id

    Year


,Player,Age,Team,First Place Votes,Total Voting Points Won,Total Voting Points Possible,% of Total Voting Points Possible That Were Won,player_id,Year
0,Malik Monk,24,SAC,0.0000,1.0000,500,0.0020,monkma01,2023


### 6 Man of the Year Award

In [209]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rank":f"{unique_table_name}Rank",
"# Tm":f"{unique_table_name}All NBA Award (e.g. 1st Team)",
"1st Tm":f"{unique_table_name}All NBA 1st Team Votes",
"2nd Tm":f"{unique_table_name}All NBA 2nd Team Votes",
"3rd Tm":f"{unique_table_name}All NBA 3rd Team Votes",                                                  
"Pos":f"{unique_table_name}Position",                         
"Player":f"{unique_table_name}Player",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"First":f"{unique_table_name}First Place Votes",
"Pts Won":f"{unique_table_name}Total Voting Points Won",
"Pts Max":f"{unique_table_name}Total Voting Points Possible",
"Share":f"{unique_table_name}% of Total Voting Points Possible That Were Won",                                                                                                                  
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
six_man_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['G', 'MP', 'PTS', 'TRB', 'AST', 'STL','BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48', 'Rank']

for item in list_of_columns_to_remove:
    six_man_df = six_man_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in six_man_df.columns:
    
    print(f"""
    {column}""")

six_man_df.head(1)


    Player

    Age

    Team

    First Place Votes

    Total Voting Points Won

    Total Voting Points Possible

    % of Total Voting Points Possible That Were Won

    player_id

    Year


,Player,Age,Team,First Place Votes,Total Voting Points Won,Total Voting Points Possible,% of Total Voting Points Possible That Were Won,player_id,Year
0,Christian Wood,27,DAL,0.0000,1.0000,500,0.0020,woodch01,2023


### DPOY Award

In [210]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rank":f"{unique_table_name}Rank",
"# Tm":f"{unique_table_name}All NBA Award (e.g. 1st Team)",
"1st Tm":f"{unique_table_name}All NBA 1st Team Votes",
"2nd Tm":f"{unique_table_name}All NBA 2nd Team Votes",
"3rd Tm":f"{unique_table_name}All NBA 3rd Team Votes",                                                  
"Pos":f"{unique_table_name}Position",                         
"Player":f"{unique_table_name}Player",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"First":f"{unique_table_name}First Place Votes",
"Pts Won":f"{unique_table_name}Total Voting Points Won",
"Pts Max":f"{unique_table_name}Total Voting Points Possible",
"Share":f"{unique_table_name}% of Total Voting Points Possible That Were Won",                                                                                                                  
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
dpoy_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['G', 'MP', 'PTS', 'TRB', 'AST', 'STL','BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48','DWS','DBPM','DRtg', 'Rank']

for item in list_of_columns_to_remove:
    dpoy_df = dpoy_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in dpoy_df.columns:
    
    print(f"""
    {column}""")

dpoy_df.head(1)


    Player

    Age

    Team

    First Place Votes

    Total Voting Points Won

    Total Voting Points Possible

    % of Total Voting Points Possible That Were Won

    player_id

    Year


,Player,Age,Team,First Place Votes,Total Voting Points Won,Total Voting Points Possible,% of Total Voting Points Possible That Were Won,player_id,Year
0,Jimmy Butler,33,MIA,0.0000,1.0000,500,0.0020,butleji01,2023


### Finals MVP Award

In [211]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rank":f"{unique_table_name}Rank",
"# Tm":f"{unique_table_name}All NBA Award (e.g. 1st Team)",
"1st Tm":f"{unique_table_name}All NBA 1st Team Votes",
"2nd Tm":f"{unique_table_name}All NBA 2nd Team Votes",
"3rd Tm":f"{unique_table_name}All NBA 3rd Team Votes",                                                  
"Pos":f"{unique_table_name}Position",                         
"Player":f"{unique_table_name}Player",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"First":f"{unique_table_name}First Place Votes",
"Pts Won":f"{unique_table_name}Total Voting Points Won",
"Pts Max":f"{unique_table_name}Total Voting Points Possible",
"Share":f"{unique_table_name}% of Total Voting Points Possible That Were Won",                                                                                                                  
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
finals_mvp.rename(name_replacement_dict,axis=1,inplace=True)

# # Get rid of dataframes as desired
list_of_columns_to_remove = ['Lg', 'G', 'MP', 'PTS', 'TRB', 'AST', 'STL','BLK', 'FG%', '3P%', 'FT%']

for item in list_of_columns_to_remove:
    finals_mvp = finals_mvp.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in finals_mvp.columns:
    
    print(f"""
    {column}""")

finals_mvp.head(1)


    Year

    Player

    Age

    Team

    player_id


,Year,Player,Age,Team,player_id
0,2023,Nikola Jokić,27,DEN,jokicni01


## TEAM REGULAR SEASON STATS

 <a class="anchor" id="Step31"></a>

### TEAM Regular Season Basic Stats

In [212]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
"GS":f"{unique_table_name}Games Started",
"MP":f"{unique_table_name}Minutes Played Per Game",
"FG":f"{unique_table_name}Field Goals Made Per Game",
"FGA":f"{unique_table_name}Field Goals Attempted Per Game",
"FG%":f"{unique_table_name}Average Field Goal % For Season",
"3P":f"{unique_table_name}3 Pointers Made Per Game",
"3PA":f"{unique_table_name}3 Point Attempts Per Game",
"3P%":f"{unique_table_name}Average 3 Point % For Season",
"2P":f"{unique_table_name}2 Pointers Made Per Game",
"2PA":f"{unique_table_name}2 Point Attempts Per Game",
"2P%":f"{unique_table_name}Average 2 Point % For Season",
"eFG%":f"{unique_table_name}Average Effective Field Goal % For Season",
"FT":f"{unique_table_name}Free Throws Made Per Game",
"FTA":f"{unique_table_name}Free Throw Attempts Per Game",
"FT%":f"{unique_table_name}Average Free Throw % For Season",
"ORB":f"{unique_table_name}Offense Rebounds Per Game",
"DRB":f"{unique_table_name}Defense Rebounds Per Game",
"TRB":f"{unique_table_name}Total Rebounds Per Game",
"AST":f"{unique_table_name}Assists Per Game",
"STL":f"{unique_table_name}Steals Per Game",
"BLK":f"{unique_table_name}Blocks Per Game",
"TOV":f"{unique_table_name}Turnovers Per Game",
"PF":f"{unique_table_name}Personal Fouls Per Game",
"PTS":f"{unique_table_name}Points Per Game",
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}
# Replace names in dataframe
nba_team_basic_stat_in_regular_season_df.rename(name_replacement_dict,axis=1,inplace=True)

# Check column names to make sure none desired to get rid of
for column in nba_team_basic_stat_in_regular_season_df.columns:
    
    print(f"""
    {column}""")

nba_team_basic_stat_in_regular_season_df.head(1)



    Rank

    Team

    Games Played

    Minutes Played Per Game

    Field Goals Made Per Game

    Field Goals Attempted Per Game

    Average Field Goal % For Season

    3 Pointers Made Per Game

    3 Point Attempts Per Game

    Average 3 Point % For Season

    2 Pointers Made Per Game

    2 Point Attempts Per Game

    Average 2 Point % For Season

    Free Throws Made Per Game

    Free Throw Attempts Per Game

    Average Free Throw % For Season

    Offense Rebounds Per Game

    Defense Rebounds Per Game

    Total Rebounds Per Game

    Assists Per Game

    Steals Per Game

    Blocks Per Game

    Turnovers Per Game

    Personal Fouls Per Game

    Points Per Game

    Year


,Rank,Team,Games Played,Minutes Played Per Game,Field Goals Made Per Game,Field Goals Attempted Per Game,Average Field Goal % For Season,3 Pointers Made Per Game,3 Point Attempts Per Game,Average 3 Point % For Season,2 Pointers Made Per Game,2 Point Attempts Per Game,Average 2 Point % For Season,Free Throws Made Per Game,Free Throw Attempts Per Game,Average Free Throw % For Season,Offense Rebounds Per Game,Defense Rebounds Per Game,Total Rebounds Per Game,Assists Per Game,Steals Per Game,Blocks Per Game,Turnovers Per Game,Personal Fouls Per Game,Points Per Game,Year
0,NaN,League Average,66,241.3000,42.3000,89.1000,0.4750,12.8000,35.0000,0.3670,29.5000,54.1000,0.5450,17.4000,22.2000,0.7830,10.6000,33.0000,43.6000,26.7000,7.4000,5.2000,13.6000,19.1000,114.8000,2024


### TEAM Regular Season Basic Opponent Stats

In [213]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
"GS":f"{unique_table_name}Games Started",
"MP":f"{unique_table_name}Minutes Played Per Game",
"FG":f"{unique_table_name}Field Goals Made Per Game",
"FGA":f"{unique_table_name}Field Goals Attempted Per Game",
"FG%":f"{unique_table_name}Average Field Goal % For Season",
"3P":f"{unique_table_name}3 Pointers Made Per Game",
"3PA":f"{unique_table_name}3 Point Attempts Per Game",
"3P%":f"{unique_table_name}Average 3 Point % For Season",
"2P":f"{unique_table_name}2 Pointers Made Per Game",
"2PA":f"{unique_table_name}2 Point Attempts Per Game",
"2P%":f"{unique_table_name}Average 2 Point % For Season",
"eFG%":f"{unique_table_name}Average Effective Field Goal % For Season",
"FT":f"{unique_table_name}Free Throws Made Per Game",
"FTA":f"{unique_table_name}Free Throw Attempts Per Game",
"FT%":f"{unique_table_name}Average Free Throw % For Season",
"ORB":f"{unique_table_name}Offense Rebounds Per Game",
"DRB":f"{unique_table_name}Defense Rebounds Per Game",
"TRB":f"{unique_table_name}Total Rebounds Per Game",
"AST":f"{unique_table_name}Assists Per Game",
"STL":f"{unique_table_name}Steals Per Game",
"BLK":f"{unique_table_name}Blocks Per Game",
"TOV":f"{unique_table_name}Turnovers Per Game",
"PF":f"{unique_table_name}Personal Fouls Per Game",
"PTS":f"{unique_table_name}Points Per Game",
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
nba_team_basic_opponent_stat_in_regular_season_df.rename(name_replacement_dict,axis=1,inplace=True)

# # Get rid of dataframes as desired
list_of_columns_to_remove = ["Rank"]

for item in list_of_columns_to_remove:
    nba_team_basic_opponent_stat_in_regular_season_df = nba_team_basic_opponent_stat_in_regular_season_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in nba_team_basic_opponent_stat_in_regular_season_df.columns:
    
    print(f"""
    {column}""")

nba_team_basic_opponent_stat_in_regular_season_df.head(1)


    Team

    Games Played

    Minutes Played Per Game

    Field Goals Made Per Game

    Field Goals Attempted Per Game

    Average Field Goal % For Season

    3 Pointers Made Per Game

    3 Point Attempts Per Game

    Average 3 Point % For Season

    2 Pointers Made Per Game

    2 Point Attempts Per Game

    Average 2 Point % For Season

    Free Throws Made Per Game

    Free Throw Attempts Per Game

    Average Free Throw % For Season

    Offense Rebounds Per Game

    Defense Rebounds Per Game

    Total Rebounds Per Game

    Assists Per Game

    Steals Per Game

    Blocks Per Game

    Turnovers Per Game

    Personal Fouls Per Game

    Points Per Game

    Year


,Team,Games Played,Minutes Played Per Game,Field Goals Made Per Game,Field Goals Attempted Per Game,Average Field Goal % For Season,3 Pointers Made Per Game,3 Point Attempts Per Game,Average 3 Point % For Season,2 Pointers Made Per Game,2 Point Attempts Per Game,Average 2 Point % For Season,Free Throws Made Per Game,Free Throw Attempts Per Game,Average Free Throw % For Season,Offense Rebounds Per Game,Defense Rebounds Per Game,Total Rebounds Per Game,Assists Per Game,Steals Per Game,Blocks Per Game,Turnovers Per Game,Personal Fouls Per Game,Points Per Game,Year
0,League Average,66,241.3000,42.3000,89.1000,0.4750,12.8000,35.0000,0.3670,29.5000,54.1000,0.5450,17.4000,22.2000,0.7830,10.6000,33.0000,43.6000,26.7000,7.4000,5.2000,13.6000,19.1000,114.8000,2024


### TEAM Regular Season Advanced Stats

In [214]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Age":f"{unique_table_name}Average Player Age",
"Team":f"{unique_table_name}Team",

"W":f"{unique_table_name}Wins",
"L":f"{unique_table_name}Losses",
"PW":f"{unique_table_name}Pythagorean Wins (expected wins based on points scored and allowed)",
"PL":f"{unique_table_name}Pythagorean Losses (expected wins based on points scored and allowed)",
"MOV":f"{unique_table_name}Margin of Victory",
"SOS":f"{unique_table_name}Strength of schedule rating (positive is above average)",
"SRS":f"{unique_table_name}Simple rating system: derived using average point differential and strength of scheduled",
"ORtg":f"{unique_table_name}Offensive Rating",
"DRtg":f"{unique_table_name}Defensive Rating",
"NRtg":f"{unique_table_name}Net Rating",
"Pace":f"{unique_table_name}Pace (Possessions per 48 Minutes)",
"FTr":f"{unique_table_name}Free Throw Attempts Per FG Attempt",
"3PAr":f"{unique_table_name}3 Point Attempts Per FG Attempt",
"TS%":f"{unique_table_name}True Shooting %",
"eFG%":f"{unique_table_name}Average Effective Field Goal % For Season",
"TOV%":f"{unique_table_name}Turnover %",
"ORB%":f"{unique_table_name}Offensive Rebounding %",  
                         
"FT/FGA":f"{unique_table_name}Free Throws Made Per FG Attempt",
"eFG%.1":f"{unique_table_name}Opponent Effective Field Goal %",
"TOV%.1":f"{unique_table_name}Opponent Turnover %",
"DRB%":f"{unique_table_name}Defensive Rebounding %",
"FT/FGA.1":f"{unique_table_name}Opponent Field Goal Throws Per FG Attempt",
"Arena":f"{unique_table_name}Arena",
"Attend.":f"{unique_table_name}Total Arena Attendance",
"Attend./G":f"{unique_table_name}Attendance Per Game",

"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
nba_advanced_team_stats.rename(name_replacement_dict,axis=1,inplace=True)

# # Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 17','Unnamed: 22','Unnamed: 27', "Rank"]

for item in list_of_columns_to_remove:
    nba_advanced_team_stats = nba_advanced_team_stats.drop(item,axis=1)


# Check column names to make sure none desired to get rid of
for column in nba_advanced_team_stats.columns:
    
    print(f"""
    {column}""")

nba_advanced_team_stats.head(1)


    Team

    Average Player Age

    Wins

    Losses

    Pythagorean Wins (expected wins based on points scored and allowed)

    Pythagorean Losses (expected wins based on points scored and allowed)

    Margin of Victory

    Strength of schedule rating (positive is above average)

    Simple rating system: derived using average point differential and strength of scheduled

    Offensive Rating

    Defensive Rating

    Net Rating

    Pace (Possessions per 48 Minutes)

    Free Throw Attempts Per FG Attempt

    3 Point Attempts Per FG Attempt

    True Shooting %

    Average Effective Field Goal % For Season

    Turnover %

    Offensive Rebounding %

    Free Throws Made Per FG Attempt

    Opponent Effective Field Goal %

    Opponent Turnover %

    Defensive Rebounding %

    Opponent Field Goal Throws Per FG Attempt

    Arena

    Total Arena Attendance

    Attendance Per Game

    Year


,Team,Average Player Age,Wins,Losses,Pythagorean Wins (expected wins based on points scored and allowed),Pythagorean Losses (expected wins based on points scored and allowed),Margin of Victory,Strength of schedule rating (positive is above average),Simple rating system: derived using average point differential and strength of scheduled,Offensive Rating,Defensive Rating,Net Rating,Pace (Possessions per 48 Minutes),Free Throw Attempts Per FG Attempt,3 Point Attempts Per FG Attempt,True Shooting %,Average Effective Field Goal % For Season,Turnover %,Offensive Rebounding %,Free Throws Made Per FG Attempt,Opponent Effective Field Goal %,Opponent Turnover %,Defensive Rebounding %,Opponent Field Goal Throws Per FG Attempt,Arena,Total Arena Attendance,Attendance Per Game,Year
0,League Average,26.6000,NaN,NaN,33,33,0.0000,0.0000,0.0000,115.6000,115.6000,NaN,98.8000,0.2490,0.3930,0.5810,0.5470,12.1000,24.3000,0.1950,0.5470,12.1000,75.7000,0.1950,NaN,605302.0000,18253.0000,2024


### TEAM Regular Season Shooting Stats

In [215]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
'MP':f'{unique_table_name}Total Minutes Played During Season',
'FG%':f'{unique_table_name}Field Goal %',
'Dist.':f'{unique_table_name}Average distance of shot',                         
'2P':f'{unique_table_name}2 Pointer Attempt Rate For Season as % of total field goals attempted',
'0-3':f'{unique_table_name}% of FGA between 0-3 ft.',
'3-10':f'{unique_table_name}% of FGA between 3-10 ft.',                         
'10-16':f'{unique_table_name}% of FGA between 10-16 ft.',
'16-3P':f'{unique_table_name}% of FGA between 16-3P ft.',
'3P':f'{unique_table_name}3 Pointer Attempt Rate For Season as % of total field goals attempted',
'2P.1':f'{unique_table_name}FG % of 2 Point Attempts',
'0-3.1':f'{unique_table_name}FG % of Attempts between 0-3 ft.',
'3-10.1':f'{unique_table_name}FG % of Attempts between 3-10 ft.',                         
'10-16.1':f'{unique_table_name}FG % of Attempts between 10-16 ft.',
'16-3P.1':f'{unique_table_name}FG % of Attempts between 16-3P ft.',
'3P.1':f'{unique_table_name}FG % of 3 Point Attempts',                       
'2P.2':f'{unique_table_name}% of Made 2 Point Fields Goals That Were Assisted',                         
'3P.2':f'{unique_table_name}% of Made 3 Point Fields Goals That Were Assisted',
'%FGA':f'{unique_table_name}% of Field Goal attempts that are dunk attempts',
'Md.':f'{unique_table_name}Number of made dunk attempts',
 
'%FGA.1':f'{unique_table_name}% of Field Goal attempts that are layups',   
'Md..1':f'{unique_table_name}# of made layups',                         
                         
'%3PA':f'{unique_table_name}% of 3 Point Attempts that come from the corner',
'3P%':f'{unique_table_name}% of 3 Point corner attempts that are made',
'Att.':f'{unique_table_name}Heave Attempts',
'Md..2':f'{unique_table_name}Number of made heaves',                     
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
nba_team_shooting_stat_in_regular_season_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 6','Unnamed: 13','Unnamed: 20','Unnamed: 23','Unnamed: 26','Unnamed: 29','Unnamed: 32', "Rank"]

for item in list_of_columns_to_remove:
    nba_team_shooting_stat_in_regular_season_df = nba_team_shooting_stat_in_regular_season_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in nba_team_shooting_stat_in_regular_season_df.columns:
    
    print(f"""
    {column}""")

nba_team_shooting_stat_in_regular_season_df.head(1)


    Team

    Games Played

    Total Minutes Played During Season

    Field Goal %

    Average distance of shot

    2 Pointer Attempt Rate For Season as % of total field goals attempted

    % of FGA between 0-3 ft.

    % of FGA between 3-10 ft.

    % of FGA between 10-16 ft.

    % of FGA between 16-3P ft.

    3 Pointer Attempt Rate For Season as % of total field goals attempted

    FG % of 2 Point Attempts

    FG % of Attempts between 0-3 ft.

    FG % of Attempts between 3-10 ft.

    FG % of Attempts between 10-16 ft.

    FG % of Attempts between 16-3P ft.

    FG % of 3 Point Attempts

    % of Made 2 Point Fields Goals That Were Assisted

    % of Made 3 Point Fields Goals That Were Assisted

    % of Field Goal attempts that are dunk attempts

    Number of made dunk attempts

    % of Field Goal attempts that are layups

    # of made layups

    % of 3 Point Attempts that come from the corner

    % of 3 Point corner attempts that are made

    Heave Attempts

    N

,Team,Games Played,Total Minutes Played During Season,Field Goal %,Average distance of shot,2 Pointer Attempt Rate For Season as % of total field goals attempted,% of FGA between 0-3 ft.,% of FGA between 3-10 ft.,% of FGA between 10-16 ft.,% of FGA between 16-3P ft.,3 Pointer Attempt Rate For Season as % of total field goals attempted,FG % of 2 Point Attempts,FG % of Attempts between 0-3 ft.,FG % of Attempts between 3-10 ft.,FG % of Attempts between 10-16 ft.,FG % of Attempts between 16-3P ft.,FG % of 3 Point Attempts,% of Made 2 Point Fields Goals That Were Assisted,% of Made 3 Point Fields Goals That Were Assisted,% of Field Goal attempts that are dunk attempts,Number of made dunk attempts,% of Field Goal attempts that are layups,# of made layups,% of 3 Point Attempts that come from the corner,% of 3 Point corner attempts that are made,Heave Attempts,Number of made heaves,Year
0,League Average,66,16008,0.4750,14.0000,0.6070,0.2430,0.2110,0.0950,0.0580,0.3930,0.5450,0.6960,0.4540,0.4430,0.4080,0.3670,0.5370,0.8440,0.0600,315,0.2760,924,0.2420,0.3950,11.4000,0.2000,2024


### TEAM Opponent Regular Season Shooting Stats

In [216]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
'MP':f'{unique_table_name}Total Minutes Played During Season',
'FG%':f'{unique_table_name}Field Goal %',
'Dist.':f'{unique_table_name}Average distance of shot',                         
'2P':f'{unique_table_name}2 Pointer Attempt Rate For Season as % of total field goals attempted',
'0-3':f'{unique_table_name}% of FGA between 0-3 ft.',
'3-10':f'{unique_table_name}% of FGA between 3-10 ft.',                         
'10-16':f'{unique_table_name}% of FGA between 10-16 ft.',
'16-3P':f'{unique_table_name}% of FGA between 16-3P ft.',
'3P':f'{unique_table_name}3 Pointer Attempt Rate For Season as % of total field goals attempted',
'2P.1':f'{unique_table_name}FG % of 2 Point Attempts',
'0-3.1':f'{unique_table_name}FG % of Attempts between 0-3 ft.',
'3-10.1':f'{unique_table_name}FG % of Attempts between 3-10 ft.',                         
'10-16.1':f'{unique_table_name}FG % of Attempts between 10-16 ft.',
'16-3P.1':f'{unique_table_name}FG % of Attempts between 16-3P ft.',
'3P.1':f'{unique_table_name}FG % of 3 Point Attempts',                       
'2P.2':f'{unique_table_name}% of Made 2 Point Fields Goals That Were Assisted',                         
'3P.2':f'{unique_table_name}% of Made 3 Point Fields Goals That Were Assisted',
'%FGA':f'{unique_table_name}% of Field Goal attempts that are dunk attempts',
'Md.':f'{unique_table_name}Number of made dunk attempts',
 
'%FGA.1':f'{unique_table_name}% of Field Goal attempts that are layups',   
'Md..1':f'{unique_table_name}# of made layups',                         
                         
'%3PA':f'{unique_table_name}% of 3 Point Attempts that come from the corner',
'3P%':f'{unique_table_name}% of 3 Point corner attempts that are made',
'Att.':f'{unique_table_name}Heave Attempts',
'Md..2':f'{unique_table_name}Number of made heaves',                     
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
nba_team_opponent_shooting_stat_in_regular_season_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 6','Unnamed: 13','Unnamed: 20','Unnamed: 23','Unnamed: 26','Unnamed: 29', "Rank"]

for item in list_of_columns_to_remove:
    nba_team_opponent_shooting_stat_in_regular_season_df = nba_team_opponent_shooting_stat_in_regular_season_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in nba_team_opponent_shooting_stat_in_regular_season_df.columns:
    
    print(f"""
    {column}""")

nba_team_opponent_shooting_stat_in_regular_season_df.head(1)


    Team

    Games Played

    Total Minutes Played During Season

    Field Goal %

    Average distance of shot

    2 Pointer Attempt Rate For Season as % of total field goals attempted

    % of FGA between 0-3 ft.

    % of FGA between 3-10 ft.

    % of FGA between 10-16 ft.

    % of FGA between 16-3P ft.

    3 Pointer Attempt Rate For Season as % of total field goals attempted

    FG % of 2 Point Attempts

    FG % of Attempts between 0-3 ft.

    FG % of Attempts between 3-10 ft.

    FG % of Attempts between 10-16 ft.

    FG % of Attempts between 16-3P ft.

    FG % of 3 Point Attempts

    % of Made 2 Point Fields Goals That Were Assisted

    % of Made 3 Point Fields Goals That Were Assisted

    % of Field Goal attempts that are dunk attempts

    Number of made dunk attempts

    % of Field Goal attempts that are layups

    # of made layups

    % of 3 Point Attempts that come from the corner

    % of 3 Point corner attempts that are made

    Year


,Team,Games Played,Total Minutes Played During Season,Field Goal %,Average distance of shot,2 Pointer Attempt Rate For Season as % of total field goals attempted,% of FGA between 0-3 ft.,% of FGA between 3-10 ft.,% of FGA between 10-16 ft.,% of FGA between 16-3P ft.,3 Pointer Attempt Rate For Season as % of total field goals attempted,FG % of 2 Point Attempts,FG % of Attempts between 0-3 ft.,FG % of Attempts between 3-10 ft.,FG % of Attempts between 10-16 ft.,FG % of Attempts between 16-3P ft.,FG % of 3 Point Attempts,% of Made 2 Point Fields Goals That Were Assisted,% of Made 3 Point Fields Goals That Were Assisted,% of Field Goal attempts that are dunk attempts,Number of made dunk attempts,% of Field Goal attempts that are layups,# of made layups,% of 3 Point Attempts that come from the corner,% of 3 Point corner attempts that are made,Year
0,League Average,66,16008,0.4750,14.0000,0.6070,0.2430,0.2110,0.0950,0.0580,0.3930,0.5450,0.6960,0.4540,0.4430,0.4080,0.3670,0.5370,0.8440,0.0600,315,0.2760,924,0.2420,0.3950,2024


### TEAM Regular Season Standings

In [217]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Team":f"{unique_table_name}Team",
"Overall":f"{unique_table_name}Overall Team Record",
"Home":f"{unique_table_name}Team Record at Home",
"Road":f"{unique_table_name}Team Record on Road",  
"Pre":f"{unique_table_name}Team Record Pre All Star Break",
"Post":f"{unique_table_name}Team Record Post All Star Break",
"≤3":f"{unique_table_name}Team Record in Games Decided by 3 points or under",
"≥10":f"{unique_table_name}Team Record in Games Decided by 10 points or over",                                                
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
regular_season_standing_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ["Rank"]

for item in list_of_columns_to_remove:
    regular_season_standing_df = regular_season_standing_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in regular_season_standing_df.columns:
    
    print(f"""
    {column}""")

regular_season_standing_df.head(1)


    Team

    Overall Team Record

    Team Record at Home

    Team Record on Road

    Team Record Pre All Star Break

    Team Record Post All Star Break

    Team Record in Games Decided by 3 points or under

    Team Record in Games Decided by 10 points or over

    Year


,Team,Overall Team Record,Team Record at Home,Team Record on Road,Team Record Pre All Star Break,Team Record Post All Star Break,Team Record in Games Decided by 3 points or under,Team Record in Games Decided by 10 points or over,Year
0,Washington Wizards,11-55,4-26,7-29,9-45,2-10,2-3,6-30,2024


## TEAM PLAYOFFS STATS

### TEAM Playoffs Basic Stats

In [218]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
"GS":f"{unique_table_name}Games Started",
"MP":f"{unique_table_name}Minutes Played Per Game",
"FG":f"{unique_table_name}Field Goals Made Per Game",
"FGA":f"{unique_table_name}Field Goals Attempted Per Game",
"FG%":f"{unique_table_name}Average Field Goal % For Season",
"3P":f"{unique_table_name}3 Pointers Made Per Game",
"3PA":f"{unique_table_name}3 Point Attempts Per Game",
"3P%":f"{unique_table_name}Average 3 Point % For Season",
"2P":f"{unique_table_name}2 Pointers Made Per Game",
"2PA":f"{unique_table_name}2 Point Attempts Per Game",
"2P%":f"{unique_table_name}Average 2 Point % For Season",
"eFG%":f"{unique_table_name}Average Effective Field Goal % For Season",
"FT":f"{unique_table_name}Free Throws Made Per Game",
"FTA":f"{unique_table_name}Free Throw Attempts Per Game",
"FT%":f"{unique_table_name}Average Free Throw % For Season",
"ORB":f"{unique_table_name}Offense Rebounds Per Game",
"DRB":f"{unique_table_name}Defense Rebounds Per Game",
"TRB":f"{unique_table_name}Total Rebounds Per Game",
"AST":f"{unique_table_name}Assists Per Game",
"STL":f"{unique_table_name}Steals Per Game",
"BLK":f"{unique_table_name}Blocks Per Game",
"TOV":f"{unique_table_name}Turnovers Per Game",
"PF":f"{unique_table_name}Personal Fouls Per Game",
"PTS":f"{unique_table_name}Points Per Game",
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
nba_team_basic_stat_in_playoffs_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ["Rank"]

for item in list_of_columns_to_remove:
    nba_team_basic_stat_in_playoffs_df = nba_team_basic_stat_in_playoffs_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in nba_team_basic_stat_in_playoffs_df.columns:
    
    print(f"""
    {column}""")

nba_team_basic_stat_in_playoffs_df.head(1)



    Team

    Games Played

    Minutes Played Per Game

    Field Goals Made Per Game

    Field Goals Attempted Per Game

    Average Field Goal % For Season

    3 Pointers Made Per Game

    3 Point Attempts Per Game

    Average 3 Point % For Season

    2 Pointers Made Per Game

    2 Point Attempts Per Game

    Average 2 Point % For Season

    Free Throws Made Per Game

    Free Throw Attempts Per Game

    Average Free Throw % For Season

    Offense Rebounds Per Game

    Defense Rebounds Per Game

    Total Rebounds Per Game

    Assists Per Game

    Steals Per Game

    Blocks Per Game

    Turnovers Per Game

    Personal Fouls Per Game

    Points Per Game

    Year


,Team,Games Played,Minutes Played Per Game,Field Goals Made Per Game,Field Goals Attempted Per Game,Average Field Goal % For Season,3 Pointers Made Per Game,3 Point Attempts Per Game,Average 3 Point % For Season,2 Pointers Made Per Game,2 Point Attempts Per Game,Average 2 Point % For Season,Free Throws Made Per Game,Free Throw Attempts Per Game,Average Free Throw % For Season,Offense Rebounds Per Game,Defense Rebounds Per Game,Total Rebounds Per Game,Assists Per Game,Steals Per Game,Blocks Per Game,Turnovers Per Game,Personal Fouls Per Game,Points Per Game,Year
0,League Average,8,241.2000,40.2000,87.2000,NaN,12.0000,34.3000,NaN,28.1000,52.9000,0.5320,17.2000,21.6000,NaN,10.3000,32.8000,43.1000,23.7000,6.8000,4.8000,12.8000,19.6000,109.6000,2023


### TEAM Playoffs Basic Opponent Stats

In [219]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
"GS":f"{unique_table_name}Games Started",
"MP":f"{unique_table_name}Minutes Played Per Game",
"FG":f"{unique_table_name}Field Goals Made Per Game",
"FGA":f"{unique_table_name}Field Goals Attempted Per Game",
"FG%":f"{unique_table_name}Average Field Goal % For Season",
"3P":f"{unique_table_name}3 Pointers Made Per Game",
"3PA":f"{unique_table_name}3 Point Attempts Per Game",
"3P%":f"{unique_table_name}Average 3 Point % For Season",
"2P":f"{unique_table_name}2 Pointers Made Per Game",
"2PA":f"{unique_table_name}2 Point Attempts Per Game",
"2P%":f"{unique_table_name}Average 2 Point % For Season",
"eFG%":f"{unique_table_name}Average Effective Field Goal % For Season",
"FT":f"{unique_table_name}Free Throws Made Per Game",
"FTA":f"{unique_table_name}Free Throw Attempts Per Game",
"FT%":f"{unique_table_name}Average Free Throw % For Season",
"ORB":f"{unique_table_name}Offense Rebounds Per Game",
"DRB":f"{unique_table_name}Defense Rebounds Per Game",
"TRB":f"{unique_table_name}Total Rebounds Per Game",
"AST":f"{unique_table_name}Assists Per Game",
"STL":f"{unique_table_name}Steals Per Game",
"BLK":f"{unique_table_name}Blocks Per Game",
"TOV":f"{unique_table_name}Turnovers Per Game",
"PF":f"{unique_table_name}Personal Fouls Per Game",
"PTS":f"{unique_table_name}Points Per Game",
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
nba_team_basic_opponent_stat_in_playoffs_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ["Rank"]

for item in list_of_columns_to_remove:
    nba_team_basic_opponent_stat_in_playoffs_df = nba_team_basic_opponent_stat_in_playoffs_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in nba_team_basic_opponent_stat_in_playoffs_df.columns:
    
    print(f"""
    {column}""")

nba_team_basic_opponent_stat_in_playoffs_df.head(1)


    Team

    Games Played

    Minutes Played Per Game

    Field Goals Made Per Game

    Field Goals Attempted Per Game

    Average Field Goal % For Season

    3 Pointers Made Per Game

    3 Point Attempts Per Game

    Average 3 Point % For Season

    2 Pointers Made Per Game

    2 Point Attempts Per Game

    Average 2 Point % For Season

    Free Throws Made Per Game

    Free Throw Attempts Per Game

    Average Free Throw % For Season

    Offense Rebounds Per Game

    Defense Rebounds Per Game

    Total Rebounds Per Game

    Assists Per Game

    Steals Per Game

    Blocks Per Game

    Turnovers Per Game

    Personal Fouls Per Game

    Points Per Game

    Year


,Team,Games Played,Minutes Played Per Game,Field Goals Made Per Game,Field Goals Attempted Per Game,Average Field Goal % For Season,3 Pointers Made Per Game,3 Point Attempts Per Game,Average 3 Point % For Season,2 Pointers Made Per Game,2 Point Attempts Per Game,Average 2 Point % For Season,Free Throws Made Per Game,Free Throw Attempts Per Game,Average Free Throw % For Season,Offense Rebounds Per Game,Defense Rebounds Per Game,Total Rebounds Per Game,Assists Per Game,Steals Per Game,Blocks Per Game,Turnovers Per Game,Personal Fouls Per Game,Points Per Game,Year
0,League Average,8,241.2000,40.2000,87.2000,NaN,12.0000,34.3000,NaN,28.1000,52.9000,0.5320,17.2000,21.6000,NaN,10.3000,32.8000,43.1000,23.7000,6.8000,4.8000,12.8000,19.6000,109.6000,2023


### TEAM Playoffs Advanced Stats

In [220]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Age":f"{unique_table_name}Average Player Age",
"Team":f"{unique_table_name}Team",

"W":f"{unique_table_name}Wins",
"L":f"{unique_table_name}Losses",
"W/L%":f"{unique_table_name}Win Loss Percentage",                         
"PW":f"{unique_table_name}Pythagorean Wins (expected wins based on points scored and allowed)",
"PL":f"{unique_table_name}Pythagorean Losses (expected wins based on points scored and allowed)",
"MOV":f"{unique_table_name}Margin of Victory",
"SOS":f"{unique_table_name}Strength of schedule rating (positive is above average)",
"SRS":f"{unique_table_name}Simple rating system: derived using average point differential and strength of scheduled",
"ORtg":f"{unique_table_name}Offensive Rating",
"DRtg":f"{unique_table_name}Defensive Rating",
"NRtg":f"{unique_table_name}Net Rating",
"Pace":f"{unique_table_name}Pace (Possessions per 48 Minutes)",
"FTr":f"{unique_table_name}Free Throw Attempts Per FG Attempt",
"3PAr":f"{unique_table_name}3 Point Attempts Per FG Attempt",
"TS%":f"{unique_table_name}True Shooting %",
"eFG%":f"{unique_table_name}Average Effective Field Goal % For Season",
"TOV%":f"{unique_table_name}Turnover %",
"ORB%":f"{unique_table_name}Offensive Rebounding %",  
                         
"FT/FGA":f"{unique_table_name}Free Throws Made Per FG Attempt",
"eFG%.1":f"{unique_table_name}Opponent Effective Field Goal %",
"TOV%.1":f"{unique_table_name}Opponent Turnover %",
"DRB%":f"{unique_table_name}Defensive Rebounding %",
"FT/FGA.1":f"{unique_table_name}Opponent Field Goal Throws Per FG Attempt",
"Arena":f"{unique_table_name}Arena",
"Attend.":f"{unique_table_name}Total Arena Attendance",
"Attend./G":f"{unique_table_name}Attendance Per Game",

"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
nba_team_advanced_stat_in_playoffs_df.rename(name_replacement_dict,axis=1,inplace=True)

# # Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 15','Unnamed: 20','Rank']

for item in list_of_columns_to_remove:
    nba_team_advanced_stat_in_playoffs_df = nba_team_advanced_stat_in_playoffs_df.drop(item,axis=1)


# Check column names to make sure none desired to get rid of
for column in nba_team_advanced_stat_in_playoffs_df.columns:
    
    print(f"""
    {column}""")

nba_team_advanced_stat_in_playoffs_df.head(1)


    Team

    Average Player Age

    Wins

    Losses

    Win Loss Percentage

    Pythagorean Wins (expected wins based on points scored and allowed)

    Pythagorean Losses (expected wins based on points scored and allowed)

    Offensive Rating

    Defensive Rating

    Net Rating

    Pace (Possessions per 48 Minutes)

    Free Throw Attempts Per FG Attempt

    3 Point Attempts Per FG Attempt

    True Shooting %

    Average Effective Field Goal % For Season

    Turnover %

    Offensive Rebounding %

    Free Throws Made Per FG Attempt

    Opponent Effective Field Goal %

    Opponent Turnover %

    Defensive Rebounding %

    Opponent Field Goal Throws Per FG Attempt

    Year


,Team,Average Player Age,Wins,Losses,Win Loss Percentage,Pythagorean Wins (expected wins based on points scored and allowed),Pythagorean Losses (expected wins based on points scored and allowed),Offensive Rating,Defensive Rating,Net Rating,Pace (Possessions per 48 Minutes),Free Throw Attempts Per FG Attempt,3 Point Attempts Per FG Attempt,True Shooting %,Average Effective Field Goal % For Season,Turnover %,Offensive Rebounding %,Free Throws Made Per FG Attempt,Opponent Effective Field Goal %,Opponent Turnover %,Defensive Rebounding %,Opponent Field Goal Throws Per FG Attempt,Year
0,League Average,27.5000,NaN,NaN,NaN,5.0000,5.0000,113.5000,113.5000,NaN,96.1000,0.2480,0.3930,0.5660,NaN,11.7000,24.0000,0.1970,NaN,11.7000,76.0000,0.1970,2023


### TEAM Playoffs Shooting Stats

In [221]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
'MP':f'{unique_table_name}Total Minutes Played During Season',
'FG%':f'{unique_table_name}Field Goal %',
'Dist.':f'{unique_table_name}Average distance of shot',                         
'2P':f'{unique_table_name}2 Pointer Attempt Rate For Season as % of total field goals attempted',
'0-3':f'{unique_table_name}% of FGA between 0-3 ft.',
'3-10':f'{unique_table_name}% of FGA between 3-10 ft.',                         
'10-16':f'{unique_table_name}% of FGA between 10-16 ft.',
'16-3P':f'{unique_table_name}% of FGA between 16-3P ft.',
'3P':f'{unique_table_name}3 Pointer Attempt Rate For Season as % of total field goals attempted',
'2P.1':f'{unique_table_name}FG % of 2 Point Attempts',
'0-3.1':f'{unique_table_name}FG % of Attempts between 0-3 ft.',
'3-10.1':f'{unique_table_name}FG % of Attempts between 3-10 ft.',                         
'10-16.1':f'{unique_table_name}FG % of Attempts between 10-16 ft.',
'16-3P.1':f'{unique_table_name}FG % of Attempts between 16-3P ft.',
'3P.1':f'{unique_table_name}FG % of 3 Point Attempts',                       
'2P.2':f'{unique_table_name}% of Made 2 Point Fields Goals That Were Assisted',                         
'3P.2':f'{unique_table_name}% of Made 3 Point Fields Goals That Were Assisted',
'%FGA':f'{unique_table_name}% of Field Goal attempts that are dunk attempts',
'Md.':f'{unique_table_name}Number of made dunk attempts',
 
'%FGA.1':f'{unique_table_name}% of Field Goal attempts that are layups',   
'Md..1':f'{unique_table_name}# of made layups',                         
                         
'%3PA':f'{unique_table_name}% of 3 Point Attempts that come from the corner',
'3P%':f'{unique_table_name}% of 3 Point corner attempts that are made',
'Att.':f'{unique_table_name}Heave Attempts',
'#':f'{unique_table_name}Number of made heaves',                     
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
nba_team_shooting_in_playoffs_df.rename(name_replacement_dict,axis=1,inplace=True)

# # Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 6','Unnamed: 13','Unnamed: 20','Unnamed: 23','Unnamed: 26','Unnamed: 29','Rank']

for item in list_of_columns_to_remove:
    nba_team_shooting_in_playoffs_df = nba_team_shooting_in_playoffs_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in nba_team_shooting_in_playoffs_df.columns:
    
    print(f"""
    {column}""")

nba_team_shooting_in_playoffs_df.head(1)


    Team

    Games Played

    Total Minutes Played During Season

    Field Goal %

    Average distance of shot

    2 Pointer Attempt Rate For Season as % of total field goals attempted

    % of FGA between 0-3 ft.

    % of FGA between 3-10 ft.

    % of FGA between 10-16 ft.

    % of FGA between 16-3P ft.

    3 Pointer Attempt Rate For Season as % of total field goals attempted

    FG % of 2 Point Attempts

    FG % of Attempts between 0-3 ft.

    FG % of Attempts between 3-10 ft.

    FG % of Attempts between 10-16 ft.

    FG % of Attempts between 16-3P ft.

    FG % of 3 Point Attempts

    % of Made 2 Point Fields Goals That Were Assisted

    % of Made 3 Point Fields Goals That Were Assisted

    % of Field Goal attempts that are dunk attempts

    Number of made dunk attempts

    % of 3 Point Attempts that come from the corner

    % of 3 Point corner attempts that are made

    Heave Attempts

    Number of made heaves

    Year


,Team,Games Played,Total Minutes Played During Season,Field Goal %,Average distance of shot,2 Pointer Attempt Rate For Season as % of total field goals attempted,% of FGA between 0-3 ft.,% of FGA between 3-10 ft.,% of FGA between 10-16 ft.,% of FGA between 16-3P ft.,3 Pointer Attempt Rate For Season as % of total field goals attempted,FG % of 2 Point Attempts,FG % of Attempts between 0-3 ft.,FG % of Attempts between 3-10 ft.,FG % of Attempts between 10-16 ft.,FG % of Attempts between 16-3P ft.,FG % of 3 Point Attempts,% of Made 2 Point Fields Goals That Were Assisted,% of Made 3 Point Fields Goals That Were Assisted,% of Field Goal attempts that are dunk attempts,Number of made dunk attempts,% of 3 Point Attempts that come from the corner,% of 3 Point corner attempts that are made,Heave Attempts,Number of made heaves,Year
0,League Average,11,2600,0.4610,14.5000,0.6070,0.1930,0.2330,0.1110,0.0700,0.3930,0.5320,0.6860,0.4760,0.4480,0.4230,0.3510,0.5010,0.8000,0.0490,41,0.2420,0.3760,2.1000,0.1000,2023


### TEAM Playoff Opponent Shooting Stats

In [222]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f"{unique_table_name}Rank",
"Player":f"{unique_table_name}Player",
"Pos":f"{unique_table_name}Position",
"Age":f"{unique_table_name}Age",
"Team":f"{unique_table_name}Team",
"G":f"{unique_table_name}Games Played",
'MP':f'{unique_table_name}Total Minutes Played During Season',
'FG%':f'{unique_table_name}Field Goal %',
'Dist.':f'{unique_table_name}Average distance of shot',                         
'2P':f'{unique_table_name}2 Pointer Attempt Rate For Season as % of total field goals attempted',
'0-3':f'{unique_table_name}% of FGA between 0-3 ft.',
'3-10':f'{unique_table_name}% of FGA between 3-10 ft.',                         
'10-16':f'{unique_table_name}% of FGA between 10-16 ft.',
'16-3P':f'{unique_table_name}% of FGA between 16-3P ft.',
'3P':f'{unique_table_name}3 Pointer Attempt Rate For Season as % of total field goals attempted',
'2P.1':f'{unique_table_name}FG % of 2 Point Attempts',
'0-3.1':f'{unique_table_name}FG % of Attempts between 0-3 ft.',
'3-10.1':f'{unique_table_name}FG % of Attempts between 3-10 ft.',                         
'10-16.1':f'{unique_table_name}FG % of Attempts between 10-16 ft.',
'16-3P.1':f'{unique_table_name}FG % of Attempts between 16-3P ft.',
'3P.1':f'{unique_table_name}FG % of 3 Point Attempts',                       
'2P.2':f'{unique_table_name}% of Made 2 Point Fields Goals That Were Assisted',                         
'3P.2':f'{unique_table_name}% of Made 3 Point Fields Goals That Were Assisted',
'%FGA':f'{unique_table_name}% of Field Goal attempts that are dunk attempts',
'Md.':f'{unique_table_name}Number of made dunk attempts',
 
'%FGA.1':f'{unique_table_name}% of Field Goal attempts that are layups',   
'Md..1':f'{unique_table_name}# of made layups',                         
                         
'%3PA':f'{unique_table_name}% of 3 Point Attempts that come from the corner',
'3P%':f'{unique_table_name}% of 3 Point corner attempts that are made',
'Att.':f'{unique_table_name}Heave Attempts',
'Md..2':f'{unique_table_name}Number of made heaves',                     
"Year":f"{unique_table_name}Year",
"Team Name Refined - accounts for players being traded":f"{unique_table_name}Team Name Refined - accounts for players being traded",
"Standardized/Modernized Team Name":f"{unique_table_name}Standardized/Modernized Team Name"}

# Replace names in dataframe
nba_team_opponent_shooting_stat_in_playoffs_df.rename(name_replacement_dict,axis=1,inplace=True)

# Get rid of dataframes as desired
list_of_columns_to_remove = ['Unnamed: 6','Unnamed: 13','Unnamed: 20','Unnamed: 23','Unnamed: 26','Rank']

for item in list_of_columns_to_remove:
    nba_team_opponent_shooting_stat_in_playoffs_df = nba_team_opponent_shooting_stat_in_playoffs_df.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in nba_team_opponent_shooting_stat_in_playoffs_df.columns:
    
    print(f"""
    {column}""")

nba_team_opponent_shooting_stat_in_playoffs_df.head(1)


    Team

    Games Played

    Total Minutes Played During Season

    Field Goal %

    Average distance of shot

    2 Pointer Attempt Rate For Season as % of total field goals attempted

    % of FGA between 0-3 ft.

    % of FGA between 3-10 ft.

    % of FGA between 10-16 ft.

    % of FGA between 16-3P ft.

    3 Pointer Attempt Rate For Season as % of total field goals attempted

    FG % of 2 Point Attempts

    FG % of Attempts between 0-3 ft.

    FG % of Attempts between 3-10 ft.

    FG % of Attempts between 10-16 ft.

    FG % of Attempts between 16-3P ft.

    FG % of 3 Point Attempts

    % of Made 2 Point Fields Goals That Were Assisted

    % of Made 3 Point Fields Goals That Were Assisted

    % of Field Goal attempts that are dunk attempts

    Number of made dunk attempts

    % of 3 Point Attempts that come from the corner

    % of 3 Point corner attempts that are made

    Year


,Team,Games Played,Total Minutes Played During Season,Field Goal %,Average distance of shot,2 Pointer Attempt Rate For Season as % of total field goals attempted,% of FGA between 0-3 ft.,% of FGA between 3-10 ft.,% of FGA between 10-16 ft.,% of FGA between 16-3P ft.,3 Pointer Attempt Rate For Season as % of total field goals attempted,FG % of 2 Point Attempts,FG % of Attempts between 0-3 ft.,FG % of Attempts between 3-10 ft.,FG % of Attempts between 10-16 ft.,FG % of Attempts between 16-3P ft.,FG % of 3 Point Attempts,% of Made 2 Point Fields Goals That Were Assisted,% of Made 3 Point Fields Goals That Were Assisted,% of Field Goal attempts that are dunk attempts,Number of made dunk attempts,% of 3 Point Attempts that come from the corner,% of 3 Point corner attempts that are made,Year
0,League Average,8,2533.0000,0.4610,14.4000,0.6070,0.1930,0.2330,0.1110,0.0700,0.3930,0.5320,0.6860,0.4760,0.4480,0.4230,0.3510,0.5010,0.8000,0.0490,41.0000,0.2420,0.3760,2023


### Team Playoff Standings

In [223]:
# Define unique name for dataframe to insert into f-string
unique_table_name = ""

# Define name replacement dict
name_replacement_dict = {"Rk":f'{unique_table_name}Rank',
                         "Team":f'{unique_table_name}Team',
                         "Overall":f'{unique_table_name}Overall Record',
                         "Home":f'{unique_table_name}Record at home',
                         "Road":f'{unique_table_name}Record on the road',
                         "E":f'{unique_table_name}Record against Eastern Conference',
                         'Year':f'{unique_table_name}Year',
                         "W":f'{unique_table_name}Record against Western Conference',
                         "Team Name Refined - accounts for players being traded":f'{unique_table_name}Team Name Refined - accounts for players being traded',
                         "Standardized/Modernized Team Name":f'{unique_table_name}Standardized/Modernized Team Name'}
                         
                         
# Replace names in dataframe
nba_team_playoff_standing.rename(name_replacement_dict,axis=1,inplace=True)

# # Get rid of dataframes as desired
# list_of_columns_to_remove = ['Unnamed: 6','Unnamed: 13','Unnamed: 20','Unnamed: 23','Unnamed: 26']

# for item in list_of_columns_to_remove:
#     nba_team_playoff_standing = nba_team_playoff_standing.drop(item,axis=1)

# Check column names to make sure none desired to get rid of
for column in nba_team_playoff_standing.columns:
    
    print(f"""
    {column}""")

nba_team_playoff_standing.head(1)                         


    Rank

    Team

    Overall Record

    Record at home

    Record on the road

    Record against Eastern Conference

    Record against Western Conference

    Year


,Rank,Team,Overall Record,Record at home,Record on the road,Record against Eastern Conference,Record against Western Conference,Year
0,20,Toronto Raptors,0-0,NaN,NaN,NaN,NaN,2023


In [224]:
# Create playoff result column
nba_team_playoff_standing['Playoff Result'] = nba_team_playoff_standing['Overall Record']

def populate_playoff_result(x):
    
    x_value = int(x.split('-')[0])
    
    y_value = int(x.split('-')[1])
    
    value_to_populate = 'FLAG'
    
    if x_value == 16:
        
        value_to_populate='Won NBA Finals'
        
    if x_value < 16 and x_value > 11:
        
        value_to_populate='Lost NBA Finals'
    
    if x_value < 12 and x_value > 7:
        
        value_to_populate='Lost in Conference Finals'
        
    if x_value < 8 and x_value > 3:
        
        value_to_populate='Lost in 2nd round'
    
    if x_value < 4:
        
        value_to_populate='Lost in 1st round'
        
    if x_value == 0:
        
        if y_value == 0:
            
            value_to_populate='Lost in play-in'
    
    return value_to_populate
    
nba_team_playoff_standing['Playoff Result'] = nba_team_playoff_standing['Playoff Result'].apply(populate_playoff_result)

nba_team_playoff_standing.head(5)

,Rank,Team,Overall Record,Record at home,Record on the road,Record against Eastern Conference,Record against Western Conference,Year,Playoff Result
0,20,Toronto Raptors,0-0,NaN,NaN,NaN,NaN,2023,Lost in play-in
1,10,Atlanta Hawks,2-4,1-2,1-2,2-4,NaN,2023,Lost in 1st round
2,1,Denver Nuggets,16-4,10-1,6-3,4-1,12-3,2023,Won NBA Finals
3,2,Miami Heat,13-10,6-4,7-6,12-6,1-4,2023,Lost NBA Finals
4,3,Boston Celtics,11-9,5-6,6-3,11-9,NaN,2023,Lost in Conference Finals


# 3C. Create player and team tables - player_id and team_id primary keys with basic name and info in the tables. Will have one to many because teams change names and locations

# Normalize Team Names (player data had city abbreviation instead of team name). Account dynamically for team name changes

In [225]:
number_of_team_cities_prior_to_transform = len(basic_player_stats_regular_season_df['Team'].unique())

In [226]:
basic_player_stats_regular_season_df[basic_player_stats_regular_season_df['Team']=='NOK']

,Player,Position,Age,Team,Games Played,Games Started,Minutes Played Per Game,Field Goals Made Per Game,Field Goals Attempted Per Game,Average Field Goal % For Season,3 Pointers Made Per Game,3 Point Attempts Per Game,Average 3 Point % For Season,2 Pointers Made Per Game,2 Point Attempts Per Game,Average 2 Point % For Season,Average Effective Field Goal % For Season,Free Throws Made Per Game,Free Throw Attempts Per Game,Average Free Throw % For Season,Offense Rebounds Per Game,Defense Rebounds Per Game,Total Rebounds Per Game,Assists Per Game,Steals Per Game,Blocks Per Game,Turnovers Per Game,Personal Fouls Per Game,Points Per Game,player_id,Season
10923,Marc Jackson,PF,32,NOK,56,25,18.3000,2.8000,6.9000,0.4100,0.0000,0.0000,0.0000,2.8000,6.9000,0.4110,0.4100,1.6000,1.8000,0.8740,1.1000,2.3000,3.4000,1.0000,0.4000,0.1000,1.2000,1.6000,7.3000,jacksma02,2007
10945,Linton Johnson,PF,26,NOK,54,0,13.3000,1.7000,3.5000,0.4890,0.2000,0.6000,0.3330,1.5000,3.0000,0.5190,0.5160,0.6000,0.7000,0.8110,1.0000,2.1000,3.0000,0.3000,0.6000,0.3000,0.5000,1.9000,4.2000,johnsli01,2007
10952,Bobby Jackson,SG,33,NOK,56,2,23.8000,3.6000,9.3000,0.3940,1.0000,2.9000,0.3270,2.7000,6.3000,0.4250,0.4460,2.3000,3.0000,0.7740,0.8000,2.4000,3.2000,2.5000,0.9000,0.1000,1.4000,1.8000,10.6000,jacksbo01,2007
10988,Brandon Bass,PF,21,NOK,21,3,7.7000,0.7000,2.1000,0.3410,0.0000,0.0000,0.0000,0.7000,2.0000,0.3490,0.3410,0.6000,0.8000,0.7500,0.6000,1.4000,2.0000,0.1000,0.1000,0.1000,0.5000,0.8000,2.0000,bassbr01,2007
11039,Hilton Armstrong,C,22,NOK,56,5,11.3000,1.2000,2.2000,0.5440,0.0000,0.0000,NaN,1.2000,2.2000,0.5440,0.5440,0.7000,1.2000,0.5970,1.0000,1.6000,2.7000,0.2000,0.2000,0.5000,0.6000,1.4000,3.1000,armsthi01,2007
11085,Devin Brown,SG,28,NOK,58,49,28.7000,4.0000,9.6000,0.4200,1.4000,3.9000,0.3570,2.6000,5.7000,0.4630,0.4930,2.1000,2.7000,0.7940,0.9000,3.4000,4.3000,2.6000,0.8000,0.2000,1.6000,2.4000,11.6000,brownde02,2007
11094,Rasual Butler,SF,27,NOK,81,38,27.4000,3.9000,9.7000,0.3980,1.7000,4.5000,0.3690,2.2000,5.2000,0.4240,0.4840,0.7000,1.1000,0.6440,0.5000,2.6000,3.2000,0.8000,0.5000,0.7000,0.8000,1.8000,10.1000,butlera01,2007
11105,Tyson Chandler,C,24,NOK,73,73,34.6000,4.0000,6.4000,0.6240,0.0000,0.0000,0.0000,4.0000,6.4000,0.6250,0.6240,1.5000,2.8000,0.5270,4.4000,8.0000,12.4000,0.9000,0.5000,1.8000,1.7000,3.3000,9.5000,chandty01,2007
11117,Cedric Simmons,PF,21,NOK,43,4,12.4000,1.3000,3.1000,0.4170,0.0000,0.0000,NaN,1.3000,3.1000,0.4170,0.4170,0.4000,0.8000,0.4850,1.0000,1.5000,2.5000,0.3000,0.2000,0.5000,0.5000,1.7000,2.9000,simmoce01,2007
11135,Peja Stojaković,SF,29,NOK,13,13,32.7000,6.4000,15.1000,0.4230,2.6000,6.5000,0.4050,3.8000,8.6000,0.4380,0.5100,2.4000,2.9000,0.8160,0.8000,3.3000,4.2000,0.8000,0.6000,0.3000,1.5000,2.2000,17.8000,stojape01,2007


In [227]:
city_name_to_team_name_dict = {'ATL':'Atlanta Hawks',
                              'BOS':'Boston Celtics',
                              'BRK':'Brooklyn Nets',
                              'CHA':'Charlotte Bobcats',
                              'CHI':'Chicago Bulls',
                              'CHO':'Charlotte Hornets',
                              'CLE':'Cleveland Cavaliers',
                              'DAL':'Dallas Mavericks',
                              'DEN':'Denver Nuggets',
                              'DET':'Detroit Pistons',
                              'GSW':'Golden State Warriors',
                              'HOU':'Houston Rockets',
                              'IND':'Indiana Pacers',
                              'LAC':'Los Angeles Clippers',
                              'LAL':'Los Angeles Lakers',
                              'MEM':'Memphis Grizzlies',
                              'MIA':'Miami Heat',
                              'MIL':'Milwaukee Bucks',
                              'MIN':'Minnesota Timberwolves',
                              'NJN':'New Jersey Nets',
                              'NOH':'New Orleans Hornets',
                              'NOK':'NO/Ok. City Hornets',
                              'NOP':'New Orleans Pelicans',
                              'NYK':'New York Knicks',
                              'OKC':'Oklahoma City Thunder',
                              'ORL':'Orlando Magic',
                              'PHI':'Philadelphia 76ers',
                              'PHO':'Phoenix Suns',
                              'POR':'Portland Trail Blazers',
                              'SAC':'Sacramento Kings',
                              'SAS':'San Antonio Spurs',
                              'SEA':'Seattle SuperSonics',
                              'TOR':'Toronto Raptors',
                              'TOT':'Multiple Teams (player traded mid season)',
                              'UTA':'Utah Jazz',
                              'WAS':'Washington Wizards'}

In [228]:
print(city_name_to_team_name_dict.values)

<built-in method values of dict object at 0x0000021EF5363F40>


In [229]:
list_of_all_dataframes = [basic_player_stats_regular_season_df,
                           advanced_player_stats_in_regular_season_df,
                           player_shooting_in_regular_season_df,
                           player_play_by_play_in_regular_season_df,
                           player_basic_playoff_stats_df,
                           player_advanced_stats_in_playoffs_df,
                           player_shooting_stats_in_playoffs_df,
                           player_play_by_play_stats_in_playoffs_df,
                           mvp_vote_df,
                           roy_vote_df,
                           all_nba_df,
                           all_defense_df,
                           mip_df,
                           six_man_df,
                           dpoy_df,
                           finals_mvp,
                          nba_team_basic_stat_in_regular_season_df,
                          nba_team_basic_opponent_stat_in_regular_season_df,
                          nba_advanced_team_stats,
                          nba_team_shooting_stat_in_regular_season_df,
                          nba_team_opponent_shooting_stat_in_regular_season_df,
                          nba_team_basic_stat_in_playoffs_df,
                          nba_team_basic_opponent_stat_in_playoffs_df,
                          nba_team_advanced_stat_in_playoffs_df,
                          nba_team_shooting_in_playoffs_df,
                          nba_team_opponent_shooting_stat_in_playoffs_df,
                          nba_team_playoff_standing,
                          regular_season_standing_df
                         ]

for i, dataframe_to_apply_dicts_to in enumerate(list_of_all_dataframes):
        
    dataframe_to_apply_dicts_to['Team'] = dataframe_to_apply_dicts_to['Team'].replace(city_name_to_team_name_dict)
    
 
    
# Assign changes from list to actual dataframes
basic_player_stats_regular_season_df,advanced_player_stats_in_regular_season_df,player_shooting_in_regular_season_df,player_play_by_play_in_regular_season_df,player_basic_playoff_stats_df,player_advanced_stats_in_playoffs_df,player_shooting_stats_in_playoffs_df,player_play_by_play_stats_in_playoffs_df,mvp_vote_df,roy_vote_df,all_nba_df,all_defense_df,mip_df,six_man_df,dpoy_df,finals_mvp,nba_team_basic_stat_in_regular_season_df,nba_team_basic_opponent_stat_in_regular_season_df,nba_advanced_team_stats,nba_team_shooting_stat_in_regular_season_df,nba_team_opponent_shooting_stat_in_regular_season_df,nba_team_basic_stat_in_playoffs_df,nba_team_basic_opponent_stat_in_playoffs_df,nba_team_advanced_stat_in_playoffs_df,nba_team_shooting_in_playoffs_df,nba_team_opponent_shooting_stat_in_playoffs_df,nba_team_playoff_standing,regular_season_standing_df = list_of_all_dataframes 

In [230]:
# Uncover all unique team names in the source data
list_of_unique_team_name_values = []


list_of_all_dataframes = [basic_player_stats_regular_season_df,
                           advanced_player_stats_in_regular_season_df,
                           player_shooting_in_regular_season_df,
                           player_play_by_play_in_regular_season_df,
                           player_basic_playoff_stats_df,
                           player_advanced_stats_in_playoffs_df,
                           player_shooting_stats_in_playoffs_df,
                           player_play_by_play_stats_in_playoffs_df,
                           mvp_vote_df,
                           roy_vote_df,
                           all_nba_df,
                           all_defense_df,
                           mip_df,
                           six_man_df,
                           dpoy_df,
                           finals_mvp,
                          nba_team_basic_stat_in_regular_season_df,
                          nba_team_basic_opponent_stat_in_regular_season_df,
                          nba_advanced_team_stats,
                          nba_team_shooting_stat_in_regular_season_df,
                          nba_team_opponent_shooting_stat_in_regular_season_df,
                          nba_team_basic_stat_in_playoffs_df,
                          nba_team_basic_opponent_stat_in_playoffs_df,
                          nba_team_advanced_stat_in_playoffs_df,
                          nba_team_shooting_in_playoffs_df,
                          nba_team_opponent_shooting_stat_in_playoffs_df,
                          nba_team_playoff_standing,
                          regular_season_standing_df
                         ]

for df in list_of_all_dataframes:
    
    unique_teams = df['Team'].unique()
    
    for ut in unique_teams:
        
        list_of_unique_team_name_values.append(ut)
        
for item in set(list_of_unique_team_name_values):
    print(item)
    
print(f"""

Unique Team Names: {len(set(list_of_unique_team_name_values))}""")

New Orleans Hornets
Minnesota Timberwolves
Houston Rockets
Phoenix Suns
Philadelphia 76ers
Indiana Pacers
Multiple Teams (player traded mid season)
New Orleans Pelicans
Orlando Magic
Boston Celtics
Toronto Raptors
Denver Nuggets
New Jersey Nets
Memphis Grizzlies
Portland Trail Blazers
Seattle SuperSonics
Brooklyn Nets
New Orleans/Oklahoma City Hornets
Chicago Bulls
Milwaukee Bucks
Dallas Mavericks
New York Knicks
Los Angeles Clippers
Cleveland Cavaliers
Miami Heat
Atlanta Hawks
League Average
Los Angeles Lakers
Sacramento Kings
San Antonio Spurs
Utah Jazz
Golden State Warriors
Charlotte Bobcats
Oklahoma City Thunder
Seattle Supersonics
Washington Wizards
Charlotte Hornets
NO/Ok. City Hornets
Detroit Pistons


Unique Team Names: 39


In [231]:
# After creating file of team_id and team name, load in and create dictionary to populate columns for all team dataframes
df = pd.read_excel(f'{source_data_path}\\Basic Team and Franchise Tables\\Dictionary File.xlsx')

df

,team_id,team_name,franchise_id
0,1,Atlanta Hawks,1
1,2,St. Louis Hawks,1
2,3,Milwaukee Hawks,1
3,4,Tri-Cities Blackhawks,1
4,5,Boston Celtics,2
5,6,Brooklyn Nets,3
6,7,New Jersey Nets,3
7,8,New York Nets,3
8,9,New York Nets,3
9,10,New Jersey Americans,3


In [232]:
team_name_to_team_id_dict = dict(zip(df['team_name'], df['team_id']))
team_name_to_franchise_id_dict = dict(zip(df['team_name'], df['franchise_id']))

list_of_all_dataframes = [basic_player_stats_regular_season_df,
                           advanced_player_stats_in_regular_season_df,
                           player_shooting_in_regular_season_df,
                           player_play_by_play_in_regular_season_df,
                           player_basic_playoff_stats_df,
                           player_advanced_stats_in_playoffs_df,
                           player_shooting_stats_in_playoffs_df,
                           player_play_by_play_stats_in_playoffs_df,
                           mvp_vote_df,
                           roy_vote_df,
                           all_nba_df,
                           all_defense_df,
                           mip_df,
                           six_man_df,
                           dpoy_df,
                           finals_mvp,
                          nba_team_basic_stat_in_regular_season_df,
                          nba_team_basic_opponent_stat_in_regular_season_df,
                          nba_advanced_team_stats,
                          nba_team_shooting_stat_in_regular_season_df,
                          nba_team_opponent_shooting_stat_in_regular_season_df,
                          nba_team_basic_stat_in_playoffs_df,
                          nba_team_basic_opponent_stat_in_playoffs_df,
                          nba_team_advanced_stat_in_playoffs_df,
                          nba_team_shooting_in_playoffs_df,
                          nba_team_opponent_shooting_stat_in_playoffs_df,
                          nba_team_playoff_standing,
                          regular_season_standing_df
                         ]

for i, dataframe_to_apply_dicts_to in enumerate(list_of_all_dataframes):
        
    dataframe_to_apply_dicts_to['team_id'] = dataframe_to_apply_dicts_to['Team'].replace(team_name_to_team_id_dict)
    dataframe_to_apply_dicts_to['franchise_id'] = dataframe_to_apply_dicts_to['Team'].replace(team_name_to_franchise_id_dict)

 
    
# Assign changes from list to actual dataframes
basic_player_stats_regular_season_df,advanced_player_stats_in_regular_season_df,player_shooting_in_regular_season_df,player_play_by_play_in_regular_season_df,player_basic_playoff_stats_df,player_advanced_stats_in_playoffs_df,player_shooting_stats_in_playoffs_df,player_play_by_play_stats_in_playoffs_df,mvp_vote_df,roy_vote_df,all_nba_df,all_defense_df,mip_df,six_man_df,dpoy_df,finals_mvp,nba_team_basic_stat_in_regular_season_df,nba_team_basic_opponent_stat_in_regular_season_df,nba_advanced_team_stats,nba_team_shooting_stat_in_regular_season_df,nba_team_opponent_shooting_stat_in_regular_season_df,nba_team_basic_stat_in_playoffs_df,nba_team_basic_opponent_stat_in_playoffs_df,nba_team_advanced_stat_in_playoffs_df,nba_team_shooting_in_playoffs_df,nba_team_opponent_shooting_stat_in_playoffs_df,nba_team_playoff_standing,regular_season_standing_df = list_of_all_dataframes 

In [233]:
# Make sure everything is reconciled
print(len(basic_player_stats_regular_season_df['team_id'].unique()))
print(len(nba_team_basic_stat_in_regular_season_df['Team'].unique()))
print(number_of_team_cities_prior_to_transform)

36
36
36


# Compile and create player table

In [234]:
def create_list_of_dataframes(folder_path,header_row_value):

    master_df_list = []

    cumulative_row_counter = 0

    for file_name in os.listdir(folder_path):
        
        if file_name == 'desktop.ini':
            
            continue

        # Create dataframe name
        dataframe_name = f'_{file_name.split(".")[0].lower().replace(" ","_")}_df'

        # Store dataframe globally with specified name
        globals()[dataframe_name] = pd.read_csv(f'{folder_path}\\{file_name}',header=header_row_value)

        # Append to master_df_list
        master_df_list.append(globals()[dataframe_name]) 

        # Cumulative_row_counter
        cumulative_row_counter = cumulative_row_counter + len(globals()[dataframe_name])
    
    globals()['master_df_list'] = master_df_list
    globals()['cumulative_row_counter'] = cumulative_row_counter
    
    return master_df_list[0].head(1)

def concat_dataframes_and_validate_and_reset_index(desired_name_of_output_dataframe,list_of_dataframes_to_concat,length_value_to_use_to_validate):
    
    globals()[desired_name_of_output_dataframe] = pd.concat(list_of_dataframes_to_concat)

    # Verify that total record count of master dataframe matches the cumulative record count of the original dataframes
    if len(globals()[desired_name_of_output_dataframe])!=length_value_to_use_to_validate:
        raise Exception('Record count of master dataframe does not match cumulative count of component dataframes')

    # Reset indices and sort
    globals()[desired_name_of_output_dataframe].reset_index(drop=True,inplace=True)
    globals()[desired_name_of_output_dataframe].head(1)

In [235]:
create_list_of_dataframes(f'{source_data_path}\\Player Stats\\Basic Player Info',0)

,Player,From,To,Pos,Ht,Wt,Birth Date,Colleges,Player-additional
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240,June 24 1968,Duke,abdelal01


In [236]:
# Validate column structure is same across all files
validate_column_structure_is_same(master_df_list)

In [237]:
# Concat/combine yearly dataframes into one AND validate that the length of the combined dataframe is the same as the sum of the yearly dataframes
concat_dataframes_and_validate_and_reset_index('basic_player_info_df',master_df_list,cumulative_row_counter)

In [238]:
# Clean player names to match cleaning done to all other tables with player name
clean_column_values(basic_player_info_df,'Player')

C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace("*",'')
C:\Users\Carey\AppData\Local\Temp\ipykernel_76228\102270080.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[column_to_clean] = dataframe[column_to_clean].str.replace(".",'')


In [239]:
# Rename columns to desired 
name_replacement_dict = {
    "Player": "full_name",
    "From": "first_year",
    "To": "last_year",
    "Pos": "positions_played",
    "Ht": "height_in_inches",
    "Wt": "weight_in_pounds",
    "Birth Date": "birth_date",
    "Colleges": "colleges_played_for",
    "Player-additional": "player_id"
}

# Replace names in dataframe
basic_player_info_df.rename(name_replacement_dict,axis=1,inplace=True)

basic_player_info_df = basic_player_info_df[['player_id','full_name', 'first_year', 'last_year', 'positions_played',
       'height_in_inches', 'weight_in_pounds', 'birth_date',
       'colleges_played_for']] 

In [240]:
def height_to_inches(height_str):
    try:
        feet, inches = height_str.split('-')
        return int(feet) * 12 + int(inches)
    except:
        return None  # or np.nan if you're using NumPy

basic_player_info_df["height_in_inches"] = basic_player_info_df["height_in_inches"].apply(height_to_inches)

def convert_to_us_date(date_str):
    try:
        return pd.to_datetime(date_str).strftime("%m/%d/%Y")
    except:
        return None  # or np.nan if using NumPy
    
basic_player_info_df["birth_date"] = basic_player_info_df["birth_date"].apply(convert_to_us_date)

In [269]:
# Manually assign one player's idkey that got changed between file extraction
basic_player_info_df.loc[basic_player_info_df['player_id'] == 'hendrta01', 'player_id'] = 'hendrita01'

In [270]:
basic_player_info_df.to_csv(f'{final_data_for_upload_path}\\Player Basic Info.csv',index=False)

# Compile, clean, and export franchise table

In [242]:
info_df = pd.read_excel(f'{source_data_path}\\Basic Team and Franchise Tables\\franchise_and_team_info.xlsx')

In [243]:
info_df

,Franchise,Lg,From,To,Franchise or Team Flag,Franchise ID,Yrs,G,W,L,W/L%,Plyfs,Div,Conf,Champ
0,Atlanta Hawks,NBA,1949-50,2024-25,Franchise,1,76.0000,6019.0000,2967.0000,3052.0000,0.4930,49.0000,12.0000,0.0000,1.0000
1,Atlanta Hawks,NBA,1968-69,2024-25,Team,1,57.0000,4601.0000,2269.0000,2332.0000,0.4930,36.0000,6.0000,0.0000,0.0000
2,St. Louis Hawks,NBA,1955-56,1967-68,Team,1,13.0000,1005.0000,553.0000,452.0000,0.5500,12.0000,6.0000,0.0000,1.0000
3,Milwaukee Hawks,NBA,1951-52,1954-55,Team,1,4.0000,281.0000,91.0000,190.0000,0.3240,0.0000,0.0000,0.0000,0.0000
4,Tri-Cities Blackhawks,NBA,1949-50,1950-51,Team,1,2.0000,132.0000,54.0000,78.0000,0.4090,1.0000,0.0000,0.0000,0.0000
5,Boston Celtics,NBA/BAA,1946-47,2024-25,Both,2,79.0000,6196.0000,3695.0000,2501.0000,0.5960,62.0000,35.0000,11.0000,18.0000
6,Brooklyn Nets,NBA/ABA,1967-68,2024-25,Franchise,3,58.0000,4694.0000,2054.0000,2640.0000,0.4380,31.0000,5.0000,2.0000,2.0000
7,Brooklyn Nets,NBA,2012-13,2024-25,Team,3,13.0000,1046.0000,472.0000,574.0000,0.4510,8.0000,0.0000,0.0000,0.0000
8,New Jersey Nets,NBA,1977-78,2011-12,Team,3,35.0000,2822.0000,1186.0000,1636.0000,0.4200,16.0000,4.0000,2.0000,0.0000
9,New York Nets,NBA,1976-77,1976-77,Team,3,1.0000,82.0000,22.0000,60.0000,0.2680,0.0000,0.0000,0.0000,0.0000


In [244]:
# Clean up table
franchises_only_df = info_df.loc[(info_df['Franchise or Team Flag']=='Franchise')|(info_df['Franchise or Team Flag']=='Both')].copy()
franchises_only_df.reset_index(drop=True)

# Change From and To to years instead of year range

def extract_first_year_plus_one(year_range):
    
    if pd.isna(year_range) or not isinstance(year_range, str):
        return None
        
    # Split the string on '-' and take the first part
    first_year = int(year_range.split('-')[0])
    
    # Add one to it
    return first_year + 1

franchises_only_df['From'] = franchises_only_df['From'].apply(extract_first_year_plus_one)
franchises_only_df['To'] = franchises_only_df['To'].apply(extract_first_year_plus_one)

# Rename columns to desired 
name_replacement_dict = {"Lg":"Leagues",
                         "From":"First Year",
                         "To":"Last Year",
                        "Yrs":"Years in Existence",
                        "G":"Total Games Played",
                        "W":"Total Wins",
                        "L":"Total Losses",
                        "W/L%":"Win-Loss Percentage",
                        "Plyfs":"Number of years team made the playoffs",
                        "Div":"Years team finished 1st or tied for 1st in divison",
                        "Conf":"Years team won the conference championship",
                        "Champ":"Years team won the league championship",
                        "Team ID":"team_name_id",
                        "Franchise ID":'franchise_id',
                         "Franchise":"franchise_name"
                        }
                             
# Replace names in dataframe
franchises_only_df.rename(name_replacement_dict,axis=1,inplace=True)

franchises_only_df = franchises_only_df[['franchise_id', 'franchise_name', 'Leagues', 'First Year', 'Last Year',
       'Years in Existence', 'Total Games Played', 'Total Wins',
       'Total Losses', 'Win-Loss Percentage',
       'Number of years team made the playoffs',
       'Years team finished 1st or tied for 1st in divison',
       'Years team won the conference championship',
       'Years team won the league championship']]

In [245]:
franchises_only_df

,franchise_id,franchise_name,Leagues,First Year,Last Year,Years in Existence,Total Games Played,Total Wins,Total Losses,Win-Loss Percentage,Number of years team made the playoffs,Years team finished 1st or tied for 1st in divison,Years team won the conference championship,Years team won the league championship
0,1,Atlanta Hawks,NBA,1950.0000,2025.0000,76.0000,6019.0000,2967.0000,3052.0000,0.4930,49.0000,12.0000,0.0000,1.0000
5,2,Boston Celtics,NBA/BAA,1947.0000,2025.0000,79.0000,6196.0000,3695.0000,2501.0000,0.5960,62.0000,35.0000,11.0000,18.0000
6,3,Brooklyn Nets,NBA/ABA,1968.0000,2025.0000,58.0000,4694.0000,2054.0000,2640.0000,0.4380,31.0000,5.0000,2.0000,2.0000
12,4,Charlotte Hornets,NBA,1989.0000,2025.0000,35.0000,2795.0000,1193.0000,1602.0000,0.4270,10.0000,0.0000,0.0000,0.0000
16,5,Chicago Bulls,NBA,1967.0000,2025.0000,59.0000,4762.0000,2422.0000,2340.0000,0.5090,36.0000,9.0000,6.0000,6.0000
17,6,Cleveland Cavaliers,NBA,1971.0000,2025.0000,55.0000,4435.0000,2096.0000,2339.0000,0.4730,25.0000,8.0000,5.0000,1.0000
18,7,Dallas Mavericks,NBA,1981.0000,2025.0000,45.0000,3625.0000,1836.0000,1789.0000,0.5060,25.0000,5.0000,3.0000,1.0000
19,8,Denver Nuggets,NBA/ABA,1968.0000,2025.0000,58.0000,4695.0000,2417.0000,2278.0000,0.5150,40.0000,13.0000,1.0000,1.0000
23,9,Detroit Pistons,NBA/BAA,1949.0000,2025.0000,77.0000,6080.0000,2871.0000,3209.0000,0.4720,43.0000,11.0000,5.0000,3.0000
26,10,Golden State Warriors,NBA/BAA,1947.0000,2025.0000,79.0000,6185.0000,3017.0000,3168.0000,0.4880,38.0000,12.0000,7.0000,7.0000


In [246]:
# Define mapping: CSV headers → DB column names
column_mapping = {
    "First Year": "first_year",
    "Last Year": "last_year",
    "Years in Existence": "years_in_existence",
    "Total Games Played": "total_games_played",
    "Total Wins": "total_wins",
    "Total Losses": "total_losses",
    "Win-Loss Percentage": "win_loss_percentage",
    "Number of years team made the playoffs": "num_playoff_appearances",
    "Years team finished 1st or tied for 1st in divison": "num_division_titles",
    "Years team won the conference championship": "num_conference_titles",
    "Years team won the league championship": "num_league_championships",
    "Leagues":'leagues'
}

# Apply the renaming
franchises_only_df.rename(columns=column_mapping, inplace=True)

franchises_only_df.to_csv(f'{final_data_for_upload_path}\\Franchise Table.csv',index=False)

In [247]:
franchises_only_df

,franchise_id,franchise_name,leagues,first_year,last_year,years_in_existence,total_games_played,total_wins,total_losses,win_loss_percentage,num_playoff_appearances,num_division_titles,num_conference_titles,num_league_championships
0,1,Atlanta Hawks,NBA,1950.0000,2025.0000,76.0000,6019.0000,2967.0000,3052.0000,0.4930,49.0000,12.0000,0.0000,1.0000
5,2,Boston Celtics,NBA/BAA,1947.0000,2025.0000,79.0000,6196.0000,3695.0000,2501.0000,0.5960,62.0000,35.0000,11.0000,18.0000
6,3,Brooklyn Nets,NBA/ABA,1968.0000,2025.0000,58.0000,4694.0000,2054.0000,2640.0000,0.4380,31.0000,5.0000,2.0000,2.0000
12,4,Charlotte Hornets,NBA,1989.0000,2025.0000,35.0000,2795.0000,1193.0000,1602.0000,0.4270,10.0000,0.0000,0.0000,0.0000
16,5,Chicago Bulls,NBA,1967.0000,2025.0000,59.0000,4762.0000,2422.0000,2340.0000,0.5090,36.0000,9.0000,6.0000,6.0000
17,6,Cleveland Cavaliers,NBA,1971.0000,2025.0000,55.0000,4435.0000,2096.0000,2339.0000,0.4730,25.0000,8.0000,5.0000,1.0000
18,7,Dallas Mavericks,NBA,1981.0000,2025.0000,45.0000,3625.0000,1836.0000,1789.0000,0.5060,25.0000,5.0000,3.0000,1.0000
19,8,Denver Nuggets,NBA/ABA,1968.0000,2025.0000,58.0000,4695.0000,2417.0000,2278.0000,0.5150,40.0000,13.0000,1.0000,1.0000
23,9,Detroit Pistons,NBA/BAA,1949.0000,2025.0000,77.0000,6080.0000,2871.0000,3209.0000,0.4720,43.0000,11.0000,5.0000,3.0000
26,10,Golden State Warriors,NBA/BAA,1947.0000,2025.0000,79.0000,6185.0000,3017.0000,3168.0000,0.4880,38.0000,12.0000,7.0000,7.0000


# Compile, clean, and export team table

In [248]:
# Clean up table
team_only_df = info_df.loc[(info_df['Franchise or Team Flag']=='Team')|(info_df['Franchise or Team Flag']=='Both')].copy()
team_only_df.reset_index(drop=True)

team_only_df['From'] = team_only_df['From'].apply(extract_first_year_plus_one)
team_only_df['To'] = team_only_df['To'].apply(extract_first_year_plus_one)

# Rename columns to desired 
name_replacement_dict = {"Lg":"Leagues",
                         "From":"First Year",
                         "To":"Last Year",
                        "Yrs":"Years in Existence",
                        "G":"Total Games Played",
                        "W":"Total Wins",
                        "L":"Total Losses",
                        "W/L%":"Win-Loss Percentage",
                        "Plyfs":"Number of years team made the playoffs",
                        "Div":"Years team finished 1st or tied for 1st in divison",
                        "Conf":"Years team won the conference championship",
                        "Champ":"Years team won the league championship",
                        "Franchise ID":'franchise_id',
                         "Franchise":'team_name'
                        }
                             
# Replace names in dataframe
team_only_df.rename(name_replacement_dict,axis=1,inplace=True)

team_only_df["team_id"] = range(1, len(team_only_df) + 1)

team_only_df = team_only_df[['team_id', 'team_name', 'Leagues', 'First Year', 'Last Year',
       'Years in Existence', 'Total Games Played', 'Total Wins',
       'Total Losses', 'Win-Loss Percentage',
       'Number of years team made the playoffs',
       'Years team finished 1st or tied for 1st in divison',
       'Years team won the conference championship',
       'Years team won the league championship','franchise_id']]

In [249]:
team_only_df

,team_id,team_name,Leagues,First Year,Last Year,Years in Existence,Total Games Played,Total Wins,Total Losses,Win-Loss Percentage,Number of years team made the playoffs,Years team finished 1st or tied for 1st in divison,Years team won the conference championship,Years team won the league championship,franchise_id
1,1,Atlanta Hawks,NBA,1969.0000,2025.0000,57.0000,4601.0000,2269.0000,2332.0000,0.4930,36.0000,6.0000,0.0000,0.0000,1
2,2,St. Louis Hawks,NBA,1956.0000,1968.0000,13.0000,1005.0000,553.0000,452.0000,0.5500,12.0000,6.0000,0.0000,1.0000,1
3,3,Milwaukee Hawks,NBA,1952.0000,1955.0000,4.0000,281.0000,91.0000,190.0000,0.3240,0.0000,0.0000,0.0000,0.0000,1
4,4,Tri-Cities Blackhawks,NBA,1950.0000,1951.0000,2.0000,132.0000,54.0000,78.0000,0.4090,1.0000,0.0000,0.0000,0.0000,1
5,5,Boston Celtics,NBA/BAA,1947.0000,2025.0000,79.0000,6196.0000,3695.0000,2501.0000,0.5960,62.0000,35.0000,11.0000,18.0000,2
7,6,Brooklyn Nets,NBA,2013.0000,2025.0000,13.0000,1046.0000,472.0000,574.0000,0.4510,8.0000,0.0000,0.0000,0.0000,3
8,7,New Jersey Nets,NBA,1978.0000,2012.0000,35.0000,2822.0000,1186.0000,1636.0000,0.4200,16.0000,4.0000,2.0000,0.0000,3
9,8,New York Nets,NBA,1977.0000,1977.0000,1.0000,82.0000,22.0000,60.0000,0.2680,0.0000,0.0000,0.0000,0.0000,3
10,9,New York Nets,ABA,1969.0000,1976.0000,8.0000,666.0000,338.0000,328.0000,0.5080,7.0000,1.0000,0.0000,2.0000,3
11,10,New Jersey Americans,ABA,1968.0000,1968.0000,1.0000,78.0000,36.0000,42.0000,0.4620,0.0000,0.0000,0.0000,0.0000,3


In [250]:
# Define mapping: CSV headers → DB column names
column_mapping = {
    "Team": "team_name",
    "Leagues": "leagues",
    "First Year": "first_year",
    "Last Year": "last_year",
    "Years in Existence": "years_in_existence",
    "Total Games Played": "total_games_played",
    "Total Wins": "total_wins",
    "Total Losses": "total_losses",
    "Win-Loss Percentage": "win_loss_percentage",
    "Number of years team made the playoffs": "num_playoff_appearances",
    "Years team finished 1st or tied for 1st in divison": "num_division_titles",
    "Years team won the conference championship": "num_conference_titles",
    "Years team won the league championship": "num_league_championships"
}

# Apply the renaming
team_only_df.rename(columns=column_mapping, inplace=True)

team_only_df.to_csv(f'{final_data_for_upload_path}\\Team Table.csv',index=False)

In [251]:
team_only_df

,team_id,team_name,leagues,first_year,last_year,years_in_existence,total_games_played,total_wins,total_losses,win_loss_percentage,num_playoff_appearances,num_division_titles,num_conference_titles,num_league_championships,franchise_id
1,1,Atlanta Hawks,NBA,1969.0000,2025.0000,57.0000,4601.0000,2269.0000,2332.0000,0.4930,36.0000,6.0000,0.0000,0.0000,1
2,2,St. Louis Hawks,NBA,1956.0000,1968.0000,13.0000,1005.0000,553.0000,452.0000,0.5500,12.0000,6.0000,0.0000,1.0000,1
3,3,Milwaukee Hawks,NBA,1952.0000,1955.0000,4.0000,281.0000,91.0000,190.0000,0.3240,0.0000,0.0000,0.0000,0.0000,1
4,4,Tri-Cities Blackhawks,NBA,1950.0000,1951.0000,2.0000,132.0000,54.0000,78.0000,0.4090,1.0000,0.0000,0.0000,0.0000,1
5,5,Boston Celtics,NBA/BAA,1947.0000,2025.0000,79.0000,6196.0000,3695.0000,2501.0000,0.5960,62.0000,35.0000,11.0000,18.0000,2
7,6,Brooklyn Nets,NBA,2013.0000,2025.0000,13.0000,1046.0000,472.0000,574.0000,0.4510,8.0000,0.0000,0.0000,0.0000,3
8,7,New Jersey Nets,NBA,1978.0000,2012.0000,35.0000,2822.0000,1186.0000,1636.0000,0.4200,16.0000,4.0000,2.0000,0.0000,3
9,8,New York Nets,NBA,1977.0000,1977.0000,1.0000,82.0000,22.0000,60.0000,0.2680,0.0000,0.0000,0.0000,0.0000,3
10,9,New York Nets,ABA,1969.0000,1976.0000,8.0000,666.0000,338.0000,328.0000,0.5080,7.0000,1.0000,0.0000,2.0000,3
11,10,New Jersey Americans,ABA,1968.0000,1968.0000,1.0000,78.0000,36.0000,42.0000,0.4620,0.0000,0.0000,0.0000,0.0000,3


# Refine and export regular season basic player stats

In [256]:
column_rename_dict = {
    "Player": "full_name",
    "Position": "position",
    "Age": "age",
    "Team": "team_name",
    "Games Played": "games_played",
    "Games Started": "games_started",
    "Minutes Played Per Game": "mpg",
    "Field Goals Made Per Game": "fgm_per_game",
    "Field Goals Attempted Per Game": "fga_per_game",
    "Average Field Goal % For Season": "fg_pct",
    "3 Pointers Made Per Game": "three_pm_per_game",
    "3 Point Attempts Per Game": "three_pa_per_game",
    "Average 3 Point % For Season": "three_pt_pct",
    "2 Pointers Made Per Game": "two_pm_per_game",
    "2 Point Attempts Per Game": "two_pa_per_game",
    "Average 2 Point % For Season": "two_pt_pct",
    "Average Effective Field Goal % For Season": "efg_pct",
    "Free Throws Made Per Game": "ftm_per_game",
    "Free Throw Attempts Per Game": "fta_per_game",
    "Average Free Throw % For Season": "ft_pct",
    "Offense Rebounds Per Game": "orb_per_game",
    "Defense Rebounds Per Game": "drb_per_game",
    "Total Rebounds Per Game": "trb_per_game",
    "Assists Per Game": "ast_per_game",
    "Steals Per Game": "stl_per_game",
    "Blocks Per Game": "blk_per_game",
    "Turnovers Per Game": "tov_per_game",
    "Personal Fouls Per Game": "pf_per_game",
    "Points Per Game": "pts_per_game",
    "player_id": "player_id",
    "Season": "season",
    "team_id": "team_id",
    "franchise_id": "franchise_id"
}

# Apply the renaming
basic_player_stats_regular_season_df.rename(columns=column_rename_dict, inplace=True)

basic_player_stats_regular_season_df 

,full_name,position,age,team_name,games_played,games_started,mpg,fgm_per_game,fga_per_game,fg_pct,three_pm_per_game,three_pa_per_game,three_pt_pct,two_pm_per_game,two_pa_per_game,two_pt_pct,efg_pct,ftm_per_game,fta_per_game,ft_pct,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,player_id,season,team_id,franchise_id
0,Ivica Zubac,C,26,Los Angeles Clippers,54,54,26.4000,4.9000,7.5000,0.6530,0.0000,0.0000,NaN,4.9000,7.5000,0.6530,0.6530,1.7000,2.5000,0.6870,2.9000,6.2000,9.2000,1.4000,0.2000,1.2000,1.2000,2.7000,11.5000,zubaciv01,2024,29,13
1,Aaron Gordon,PF,28,Denver Nuggets,61,61,31.3000,5.5000,9.8000,0.5600,0.5000,1.9000,0.2890,5.0000,7.9000,0.6240,0.5880,2.3000,3.5000,0.6470,2.3000,4.1000,6.4000,3.3000,0.8000,0.6000,1.4000,1.9000,13.8000,gordoaa01,2024,18,8
2,Jeff Green,PF,37,Houston Rockets,62,5,16.2000,1.9000,4.2000,0.4570,0.5000,1.5000,0.3120,1.4000,2.7000,0.5390,0.5140,1.6000,2.0000,0.8080,0.6000,1.5000,2.1000,0.9000,0.2000,0.4000,0.7000,1.3000,5.9000,greenje02,2024,25,11
3,Jalen Green,SG,21,Houston Rockets,66,66,31.0000,6.5000,15.7000,0.4160,2.2000,6.8000,0.3190,4.3000,8.8000,0.4920,0.4860,3.4000,4.2000,0.8090,0.4000,4.4000,4.8000,3.3000,0.7000,0.3000,2.2000,1.3000,18.6000,greenja05,2024,25,11
4,Draymond Green,PF,33,Golden State Warriors,40,37,26.5000,3.4000,6.9000,0.4950,1.0000,2.4000,0.4230,2.4000,4.5000,0.5330,0.5690,1.2000,1.5000,0.7540,1.3000,5.7000,7.0000,5.9000,0.8000,0.9000,2.5000,3.1000,9.0000,greendr01,2024,22,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,Greg Ostertag,C,30,Utah Jazz,78,51,27.6000,2.7000,5.6000,0.4760,0.0000,0.0000,0.0000,2.7000,5.6000,0.4770,0.4760,1.4000,2.5000,0.5790,2.8000,4.6000,7.4000,1.6000,0.4000,1.8000,1.3000,2.9000,6.8000,ostergr01,2004,60,29
13101,Michael Olowokandi,C,28,Minnesota Timberwolves,43,25,21.5000,2.8000,6.6000,0.4250,0.0000,0.0000,NaN,2.8000,6.6000,0.4250,0.4250,0.8000,1.4000,0.5900,1.8000,3.9000,5.7000,0.6000,0.4000,1.6000,1.3000,3.2000,6.5000,olowomi01,2004,38,18
13102,Kevin Ollie,PG,31,Cleveland Cavaliers,82,7,17.1000,1.2000,3.1000,0.3700,0.0000,0.1000,0.4440,1.1000,3.0000,0.3670,0.3770,1.8000,2.1000,0.8350,0.3000,1.8000,2.1000,2.9000,0.6000,0.1000,1.0000,1.5000,4.2000,ollieke01,2004,15,6
13103,Mehmet Okur,PF,24,Detroit Pistons,71,33,22.3000,3.5000,7.6000,0.4630,0.3000,0.7000,0.3750,3.3000,7.0000,0.4720,0.4800,2.3000,2.9000,0.7750,2.3000,3.7000,5.9000,1.0000,0.5000,0.9000,1.4000,1.9000,9.6000,okurme01,2004,20,9


In [265]:
# make sure player_id is not null and make sure it matches one in the player table
print(len(basic_player_stats_regular_season_df.loc[basic_player_stats_regular_season_df['player_id'].isnull()]))

# print any player_id in basic_player_stats_regular_season_df not in basic_player_info_df's
missing_ids = basic_player_stats_regular_season_df[~basic_player_stats_regular_season_df['player_id'].isin(basic_player_info_df['player_id'])]

# reassign id
basic_player_stats_regular_season_df.loc[147,'player_id']


0


,full_name,position,age,team_name,games_played,games_started,mpg,fgm_per_game,fga_per_game,fg_pct,three_pm_per_game,three_pa_per_game,three_pt_pct,two_pm_per_game,two_pa_per_game,two_pt_pct,efg_pct,ftm_per_game,fta_per_game,ft_pct,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,player_id,season,team_id,franchise_id
147,Taylor Hendricks,PF,20,Utah Jazz,24,7,17.5000,2.2000,5.1000,0.4230,1.0000,3.0000,0.3240,1.2000,2.2000,0.5580,0.5160,0.4000,0.5000,0.7690,1.2000,3.0000,4.2000,0.5000,0.6000,0.7000,0.7000,1.9000,5.7000,hendrita01,2024,60,29


In [257]:
basic_player_stats_regular_season_df.to_csv(f'{final_data_for_upload_path}\\Basic Player Stats Regular Season Table.csv',index=False)